In [1]:
import warnings
warnings.filterwarnings('ignore')
from numpy import set_printoptions
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
pd.options.display.max_columns=None
import seaborn as sns 
from pandas import read_csv
import io
import base64
import json
from openpyxl import Workbook
from openpyxl.drawing.image import Image as XLImage

import os
import os.path
from unidecode import unidecode

from urllib.error import HTTPError

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from sklearn.model_selection import StratifiedKFold

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from mlens.ensemble import SuperLearner


from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import normalized_mutual_info_score
from sklearn.metrics import cohen_kappa_score


[MLENS] backend: threading


### CARGUE DE DATOS Y CONEXIÓN A DRIVE

In [2]:
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/credentials.json', SCOPES) # Reemplazar con la ruta correcta
        creds = flow.run_local_server(port=0)
    with open('C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/token.json', 'w') as token:
        token.write(creds.to_json())
        
# Crear una instancia de la API de Drive
drive_service = build('drive', 'v3', credentials=creds)

# ID de la carpeta de Google Drive
folder_id = '1hQeetmO4XIObUefS_nzePqKqq3VksUEC'

# Ruta de destino para guardar los archivos descargados
save_path = 'C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/UNIVERSIDAD-DISTRITAL-PROYECTO/MÓDULO ANALÍTICA PREDICTIVA/DATOS'  # Reemplazar con la ruta deseada

# Función para descargar archivos de la carpeta de Drive
def download_folder(folder_id, save_path):
    results = drive_service.files().list(
        q=f"'{folder_id}' in parents and trashed=false",
        fields='files(id, name)').execute()
    items = results.get('files', [])
    for item in items:
        file_id = item['id']
        file_name = item['name']
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.FileIO(os.path.join(save_path, file_name), 'wb')
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Descargando {file_name}: {int(status.progress() * 100)}%")
    print("Descarga completa")

# Descargar archivos de la carpeta
download_folder(folder_id, save_path)

# Listar archivos descargados
files = os.listdir(save_path)
print("Archivos descargados:")
for file in files:
    print(file)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=899973764197-biu188dkvsgi2al0fh29udm7keak0lh0.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53297%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=19CEAml8YPwgYaYzLezJZZuxjMyD9L&access_type=offline
Descargando Limpieza de datos 1 UD.ipynb: 100%
Descargando industrial10.csv: 100%
Descargando industrial9.csv: 100%
Descargando industrial8.csv: 100%
Descargando industrial7.csv: 100%
Descargando industrial6.csv: 100%
Descargando industrial5.csv: 100%
Descargando industrial3.csv: 100%
Descargando industrial4.csv: 100%
Descargando industrial2.csv: 100%
Descargando industrial1.csv: 100%
Descarga completa
Archivos descargados:
industrial1.csv
industrial10.csv
industrial2.csv
industrial3.csv
industrial4.csv
industrial5.csv
industrial6.csv
industrial7.csv
industrial8.csv
industrial9.csv
Limpieza de datos 1 UD.ipynb


In [3]:
"""
variables_por_carrera = {
    'industrial': {
        '1': ['PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES','QUIMICA_ICFES','IDIOMA_ICFES','LOCALIDAD','RENDIMIENTO_UNO', 'PROMEDIO_UNO'],
        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'BIOLOGIA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'PROMEDIO_UNO', 'CAR_UNO', 'NCC_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_QUIMICA', 'NOTA_CFJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_EE_UNO','RENDIMIENTO_DOS', 'PROMEDIO_DOS'],
        '3': ['PROMEDIO_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_TEXTOS', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NCC_DOS', 'NCA_DOS', 'NAA_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_PBASICA', 'NOTA_EE_DOS', 'RENDIMIENTO_TRES', 'PROMEDIO_TRES'],
        '4': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ESTADISTICA_UNO', 'NOTA_TERMODINAMICA', 'NOTA_TGS', 'NOTA_EE_TRES','RENDIMIENTO_CUATRO', 'PROMEDIO_CUATRO'],
        '5': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS','PROMEDIO_TRES', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_TERMODINAMICA', 'NOTA_ECUACIONES', 'NOTA_ESTADISTICA_DOS', 'NOTA_FISICA_DOS', 'NOTA_MECANICA', 'NOTA_PROCESOSQ', 'RENDIMIENTO_CINCO', 'RENDIMIENTO_CINCO'],
        '6': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_ADMINISTRACION', 'NOTA_LENGUA_UNO', 'NOTA_EI_UNO', 'NOTA_EI_DOS', 'RENDIMIENTO_SEIS', 'PROMEDIO_SEIS'],
        '7': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_LENGUA_UNO', 'NOTA_EI_DOS', 'PROMEDIO_SEIS', 'NCA_SEIS', 'NOTA_PLINEAL', 'NOTA_DISE O', 'NOTA_EI_TRES','RENDIMIENTO_SIETE', 'PROMEDIO_SIETE'],
        '8': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_LENGUA_UNO','PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'RENDIMIENTO_OCHO', 'PROMEDIO_OCHO'],
        '9': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'PROMEDIO_OCHO', 'NCC_OCHO', 'NOTA_LOG_UNO', 'NOTA_GOPERACIONES','NOTA_CALIDAD_DOS', 'NOTA_LENGUA_DOS', 'NOTA_CONTEXTO', 'RENDIMIENTO_NUEVE', 'PROMEDIO_NUEVE'],
        '10': ['PROMEDIO_SEIS', 'PROMEDIO_SIETE', 'PROMEDIO_OCHO', 'NOTA_CALIDAD_DOS', 'PROMEDIO_NUEVE', 'NAA_NUEVE', 'NOTA_GRADO_UNO', 'NOTA_LOG_DOS', 'NOTA_FINANZAS', 'NOTA_HISTORIA', 'RENDIMIENTO_DIEZ', 'PROMEDIO_DIEZ']
    },
    'sistemas': {
        '1': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'QUIMICA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'RENDIMIENTO_UNO', 'PROMEDIO_UNO'],
        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'NOTA_DIFERENCIAL', 'NOTA_PROG_BASICA', 'NOTA_CATEDRA_FJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_CATEDRA_DEM', 'NOTA_CATEDRA_CON', 'NOTA_LOGICA', 'RENDIMIENTO_DOS', 'PROMEDIO_DOS'],
        '3': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_PROG_BASICA', 'NOTA_TEXTOS', 'NOTA_CATEDRA_DEM', 'NOTA_INTEGRAL', 'NOTA_PROG_ORIENTADA', 'NOTA_ETICA', 'RENDIMIENTO_TRES', 'PROMEDIO_TRES'],
        '4': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'PROMEDIO_DOS', 'PROMEDIO_TRES', 'NOTA_PROG_BASICA', 'NOTA_TEXTOS', 'NOTA_CATEDRA_DEM', 'NOTA_INTEGRAL', 'NOTA_PROG_ORIENTADA', 'NOTA_ETICA', 'NOTA_FISICA_DOS', 'NOTA_TGS', 'NOTA_PROG_AVANZADA', 'RENDIMIENTO_CUATRO', 'PROMEDIO_CUATRO'],
        '5': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_TERMODINAMICA', 'NOTA_ECUACIONES', 'NOTA_ESTADISTICA_DOS', 'NOTA_FISICA_DOS', 'NOTA_MECANICA', 'NOTA_PROCESOSQ', 'RENDIMIENTO_CINCO', 'PROMEDIO_CINCO'],
        '6': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ECONOMIA_UNO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_ADMINISTRACION', 'NOTA_LENGUA_UNO', 'NOTA_EI_UNO', 'NOTA_EI_DOS', 'RENDIMIENTO_SEIS', 'PROMEDIO_SEIS'],
        '7': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_LENGUA_UNO', 'NOTA_EI_DOS', 'PROMEDIO_SEIS', 'NCA_SEIS', 'NOTA_PLINEAL', 'NOTA_DISE O', 'NOTA_EI_TRES', 'RENDIMIENTO_SIETE', 'PROMEDIO_SIETE'],
        '8': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_LENGUA_UNO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'RENDIMIENTO_OCHO', 'PROMEDIO_OCHO'],
        '9': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'PROMEDIO_OCHO', 'NCC_OCHO', 'NOTA_LOG_UNO', 'NOTA_GOPERACIONES', 'NOTA_CALIDAD_DOS', 'NOTA_LENGUA_DOS', 'NOTA_CONTEXTO', 'RENDIMIENTO_NUEVE', 'PROMEDIO_NUEVE'],
        '10': ['PROMEDIO_SEIS', 'PROMEDIO_SIETE', 'PROMEDIO_OCHO', 'NOTA_CALIDAD_DOS', 'PROMEDIO_NUEVE', 'NAA_NUEVE', 'NOTA_GRADO_UNO', 'NOTA_LOG_DOS', 'NOTA_FINANZAS', 'NOTA_HISTORIA', 'RENDIMIENTO_DIEZ', 'PROMEDIO_DIEZ']
    },
    'catastral': {
        '1': ['variable1_catastral', 'variable2_catastral', 'variable3_catastral'],
        '2': ['variable4_catastral', 'variable5_catastral', 'variable6_catastral']
    }
}
"""

"\nvariables_por_carrera = {\n    'industrial': {\n        '1': ['PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES','QUIMICA_ICFES','IDIOMA_ICFES','LOCALIDAD','RENDIMIENTO_UNO', 'PROMEDIO_UNO'],\n        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'BIOLOGIA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'PROMEDIO_UNO', 'CAR_UNO', 'NCC_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_QUIMICA', 'NOTA_CFJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_EE_UNO','RENDIMIENTO_DOS', 'PROMEDIO_DOS'],\n        '3': ['PROMEDIO_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_TEXTOS', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NCC_DOS', 'NCA_DOS', 'NAA_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_PBASICA', 'NOTA_EE_DOS', 'RENDIMIENTO_TRES', 'PROMEDIO_TRES'],\n        '4': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ESTADISTICA_UNO', 'NOTA_TERMODINAMICA', 'NO

In [4]:
variables_por_carrera = {
    'industrial': {
        '1': ['PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES','QUIMICA_ICFES','IDIOMA_ICFES','LOCALIDAD','RENDIMIENTO_UNO'],
        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'BIOLOGIA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'PROMEDIO_UNO', 'CAR_UNO', 'NCC_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_QUIMICA', 'NOTA_CFJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_EE_UNO','RENDIMIENTO_DOS'],
        '3': ['PROMEDIO_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_TEXTOS', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NCC_DOS', 'NCA_DOS', 'NAA_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_PBASICA', 'NOTA_EE_DOS', 'RENDIMIENTO_TRES'],
        '4': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ESTADISTICA_UNO', 'NOTA_TERMODINAMICA', 'NOTA_TGS', 'NOTA_EE_TRES','RENDIMIENTO_CUATRO'],
        '5': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS','PROMEDIO_TRES', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_TERMODINAMICA', 'NOTA_ECUACIONES', 'NOTA_ESTADISTICA_DOS', 'NOTA_FISICA_DOS', 'NOTA_MECANICA', 'NOTA_PROCESOSQ', 'RENDIMIENTO_CINCO'],
        '6': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_ADMINISTRACION', 'NOTA_LENGUA_UNO', 'NOTA_EI_UNO', 'NOTA_EI_DOS', 'RENDIMIENTO_SEIS'],
        '7': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_LENGUA_UNO', 'NOTA_EI_DOS', 'PROMEDIO_SEIS', 'NCA_SEIS', 'NOTA_PLINEAL', 'NOTA_DISE O', 'NOTA_EI_TRES','RENDIMIENTO_SIETE'],
        '8': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_LENGUA_UNO','PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'RENDIMIENTO_OCHO'],
        '9': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'PROMEDIO_OCHO', 'NCC_OCHO', 'NOTA_LOG_UNO', 'NOTA_GOPERACIONES','NOTA_CALIDAD_DOS', 'NOTA_LENGUA_DOS', 'NOTA_CONTEXTO', 'RENDIMIENTO_NUEVE'],
        '10': ['PROMEDIO_SEIS', 'PROMEDIO_SIETE', 'PROMEDIO_OCHO', 'NOTA_CALIDAD_DOS', 'PROMEDIO_NUEVE', 'NAA_NUEVE', 'NOTA_GRADO_UNO', 'NOTA_LOG_DOS', 'NOTA_FINANZAS', 'NOTA_HISTORIA', 'RENDIMIENTO_DIEZ']
    },
    'sistemas': {
        '1': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'QUIMICA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'RENDIMIENTO_UNO'],
        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'NOTA_DIFERENCIAL', 'NOTA_PROG_BASICA', 'NOTA_CATEDRA_FJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_CATEDRA_DEM', 'NOTA_CATEDRA_CON', 'NOTA_LOGICA', 'RENDIMIENTO_DOS'],
        '3': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_PROG_BASICA', 'NOTA_TEXTOS', 'NOTA_CATEDRA_DEM', 'NOTA_INTEGRAL', 'NOTA_PROG_ORIENTADA', 'NOTA_ETICA', 'RENDIMIENTO_TRES'],
        '4': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'PROMEDIO_DOS', 'PROMEDIO_TRES', 'NOTA_PROG_BASICA', 'NOTA_TEXTOS', 'NOTA_CATEDRA_DEM', 'NOTA_INTEGRAL', 'NOTA_PROG_ORIENTADA', 'NOTA_ETICA', 'NOTA_FISICA_DOS', 'NOTA_TGS', 'NOTA_PROG_AVANZADA', 'RENDIMIENTO_CUATRO'],
        '5': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_TERMODINAMICA', 'NOTA_ECUACIONES', 'NOTA_ESTADISTICA_DOS', 'NOTA_FISICA_DOS', 'NOTA_MECANICA', 'NOTA_PROCESOSQ', 'RENDIMIENTO_CINCO'],
        '6': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ECONOMIA_UNO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_ADMINISTRACION', 'NOTA_LENGUA_UNO', 'NOTA_EI_UNO', 'NOTA_EI_DOS', 'RENDIMIENTO_SEIS'],
        '7': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_LENGUA_UNO', 'NOTA_EI_DOS', 'PROMEDIO_SEIS', 'NCA_SEIS', 'NOTA_PLINEAL', 'NOTA_DISE O', 'NOTA_EI_TRES', 'RENDIMIENTO_SIETE'],
        '8': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_LENGUA_UNO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'RENDIMIENTO_OCHO'],
        '9': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'PROMEDIO_OCHO', 'NCC_OCHO', 'NOTA_LOG_UNO', 'NOTA_GOPERACIONES', 'NOTA_CALIDAD_DOS', 'NOTA_LENGUA_DOS', 'NOTA_CONTEXTO', 'RENDIMIENTO_NUEVE'],
        '10': ['PROMEDIO_SEIS', 'PROMEDIO_SIETE', 'PROMEDIO_OCHO', 'NOTA_CALIDAD_DOS', 'PROMEDIO_NUEVE', 'NAA_NUEVE', 'NOTA_GRADO_UNO', 'NOTA_LOG_DOS', 'NOTA_FINANZAS', 'NOTA_HISTORIA', 'RENDIMIENTO_DIEZ']
    },
    'catastral': {
        '1': ['variable1_catastral', 'variable2_catastral', 'variable3_catastral'],
        '2': ['variable4_catastral', 'variable5_catastral', 'variable6_catastral']
    }
}

In [5]:
def cargar_datos(carrera, semestre):
    
    ruta_archivo = f'C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/UNIVERSIDAD-DISTRITAL-PROYECTO/MÓDULO ANALÍTICA PREDICTIVA/DATOS/{carrera}{semestre}.csv'
    datos = pd.read_csv(ruta_archivo,sep=";")
    
    return datos
carrera="industrial"
semestre="1"
#modelo="CatBoostClassifier"
modelos_seleccionados = ['KNeighbors', 'RedNeuronal']  # Lista de modelos que quieres incluir


In [6]:
"""
array(['KNeighborsClassifier', 'SVC', 'Decision Tree', 'Naive Bayes',
       'LDA', 'BaggingClassifier', 'RandomForestClassifier',
       'ExtraTreesClassifier', 'AdaBoostClassifier',
       'GradientBoostingClassifier', 'XGBClassifier', ' XGBClassifier',
       'CatBoostClassifier', 'LGBMClassifier', 'VotingClassifier_hard',
       'VotingClassifier_soft', 'StackingClassifier_lineal',
       'StackingClassifier_nolineal', 'VotingClassifier_weighted',
       'Super_aprendiz', 'Super_Aprendiz', 'Super_aprendiz_dos_capas',
       'Super_Aprendiz_dos_capas', 'Red_Neuronal'], dtype=object)"""

"\narray(['KNeighborsClassifier', 'SVC', 'Decision Tree', 'Naive Bayes',\n       'LDA', 'BaggingClassifier', 'RandomForestClassifier',\n       'ExtraTreesClassifier', 'AdaBoostClassifier',\n       'GradientBoostingClassifier', 'XGBClassifier', ' XGBClassifier',\n       'CatBoostClassifier', 'LGBMClassifier', 'VotingClassifier_hard',\n       'VotingClassifier_soft', 'StackingClassifier_lineal',\n       'StackingClassifier_nolineal', 'VotingClassifier_weighted',\n       'Super_aprendiz', 'Super_Aprendiz', 'Super_aprendiz_dos_capas',\n       'Super_Aprendiz_dos_capas', 'Red_Neuronal'], dtype=object)"

In [7]:
datos = cargar_datos(carrera, semestre)
columnas_filtradas = variables_por_carrera[carrera][semestre]
df = datos[columnas_filtradas]
print("DataFrame con columnas filtradas:")
df=df.astype(int)
df

DataFrame con columnas filtradas:


,PG_ICFES,CON_MAT_ICFES,FISICA_ICFES,QUIMICA_ICFES,IDIOMA_ICFES,LOCALIDAD,RENDIMIENTO_UNO
0,474,65,73,75,79,20,2
1,523,99,69,78,60,1,2
2,483,63,76,82,77,20,1
3,528,88,69,78,75,19,1
4,478,78,69,66,69,8,2
...,...,...,...,...,...,...,...
2003,342,68,0,0,58,11,2
2004,347,69,0,0,71,19,2
2005,348,67,0,0,68,11,1
2006,340,70,0,0,63,11,2


In [8]:
def numero_a_letras(numero):
    numeros_letras = ['cero', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis', 'siete', 'ocho', 'nueve', 'diez']
    return numeros_letras[int(numero)]

semestre_en_letras = numero_a_letras(semestre)
print(semestre_en_letras)


uno


In [9]:
X = df.loc[:, ~df.columns.str.contains(f'RENDIMIENTO_{semestre_en_letras.upper()}')]
Y = df.loc[:, df.columns.str.contains(f'RENDIMIENTO_{semestre_en_letras.upper()}')]                                                     
print("Separación de datos usando Pandas") 
print(X.shape, Y.shape)

Separación de datos usando Pandas
(2008, 6) (2008, 1)


In [10]:
#X = X.astype('float32')                         
Y = LabelEncoder().fit_transform(Y.astype('str'))                
print(X.shape, Y.shape)

(2008, 6) (2008,)


In [11]:
from collections import Counter
frecuencias = Counter(Y)
for valor, frecuencia in frecuencias.items():
    print(f"El valor {valor} aparece {frecuencia} veces")

El valor 1 aparece 1249 veces
El valor 0 aparece 675 veces
El valor 2 aparece 83 veces
El valor 3 aparece 1 veces


### TRANSFORMACIÓN DE DATOS

In [12]:
X_T_JOHNSON1 = X.copy(deep=True)
def transformacion_johnson(X):
    transformador_johnson = PowerTransformer(method='yeo-johnson', standardize=True).fit(X)
    datos_transformados = transformador_johnson.transform(X)
    set_printoptions(precision=3)
    print(datos_transformados[:5, :])
    datos_transformados_df = pd.DataFrame(data=datos_transformados, columns=X.columns)
    return datos_transformados_df
Xpandas_T_JOHNSON1 = transformacion_johnson(X_T_JOHNSON1)
Xpandas_T_JOHNSON1.head(2)

[[ 1.598 -0.897  1.005  1.046  1.268  1.353]
 [ 2.502  2.983  0.97   1.072 -0.304 -2.363]
 [ 1.767 -1.135  1.031  1.104  1.099  1.353]
 [ 2.592  1.758  0.97   1.072  0.931  1.202]
 [ 1.673  0.621  0.97   0.965  0.432 -0.682]]


,PG_ICFES,CON_MAT_ICFES,FISICA_ICFES,QUIMICA_ICFES,IDIOMA_ICFES,LOCALIDAD
0,1.597977,-0.896979,1.005474,1.046461,1.268136,1.353466
1,2.501943,2.982940,0.970095,1.071635,-0.304403,-2.362712


In [13]:
"""
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Crear una instancia de RandomOverSampler
ros = RandomOverSampler(sampling_strategy='auto', random_state=42)

# Aplicar RandomOverSampler para generar muestras sintéticas de las clases minoritarias
Xpandas_T_JOHNSON1, Y = ros.fit_resample(Xpandas_T_JOHNSON1, Y)

# Imprimir la distribución de clases después del oversampling
print("Distribución de clases después del oversampling:", Counter(Y))
"""


'\nfrom imblearn.over_sampling import RandomOverSampler\nfrom sklearn.model_selection import train_test_split\n\n# Crear una instancia de RandomOverSampler\nros = RandomOverSampler(sampling_strategy=\'auto\', random_state=42)\n\n# Aplicar RandomOverSampler para generar muestras sintéticas de las clases minoritarias\nXpandas_T_JOHNSON1, Y = ros.fit_resample(Xpandas_T_JOHNSON1, Y)\n\n# Imprimir la distribución de clases después del oversampling\nprint("Distribución de clases después del oversampling:", Counter(Y))\n'

In [14]:
X_trn, X_tst, Y_trn, Y_tst = train_test_split(Xpandas_T_JOHNSON1, Y, test_size=0.3, random_state=2)
print('DATOS: Son {} datos para entrenamiento y {} datos para prueba'.format(X_trn.shape[0], X_tst.shape[0]))

DATOS: Son 1405 datos para entrenamiento y 603 datos para prueba


In [15]:
"""
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Crear una instancia de RandomOverSampler
ros = RandomOverSampler(sampling_strategy='auto', random_state=42)

# Aplicar RandomOverSampler para generar muestras sintéticas de las clases minoritarias
X_trn,Y_trn = ros.fit_resample(X_trn, Y_trn)

# Imprimir la distribución de clases después del oversampling
print("Distribución de clases después del oversampling:", Counter(Y_trn))

"""

'\nfrom imblearn.over_sampling import RandomOverSampler\nfrom sklearn.model_selection import train_test_split\n\n# Crear una instancia de RandomOverSampler\nros = RandomOverSampler(sampling_strategy=\'auto\', random_state=42)\n\n# Aplicar RandomOverSampler para generar muestras sintéticas de las clases minoritarias\nX_trn,Y_trn = ros.fit_resample(X_trn, Y_trn)\n\n# Imprimir la distribución de clases después del oversampling\nprint("Distribución de clases después del oversampling:", Counter(Y_trn))\n\n'

In [16]:
from collections import Counter
frecuencias = Counter(Y_trn)   
for valor, frecuencia in frecuencias.items():
    print(f"El valor {valor} aparece {frecuencia} veces")   

El valor 1 aparece 861 veces
El valor 0 aparece 477 veces
El valor 2 aparece 66 veces
El valor 3 aparece 1 veces


### KNEIGHBORSCLASSIFIER

In [17]:
def entrenar_modelo_knn_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {
        'n_neighbors': [i for i in range(14, 18, 1)],
        'metric': ['euclidean', 'manhattan', 'minkowski'],
        'algorithm': ['auto'],
        'p': [i for i in range(1, 6)],
        'weights': ['uniform']
    }
    modelo = KNeighborsClassifier()
    semilla = 5
    num_folds = 10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn_transformado, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)

    # Entrenar el modelo con los mejores hiperparámetros
    mejor_modelo = KNeighborsClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_knn = entrenar_modelo_knn_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 60.427558257345495
Mejor valor PARAMETRO usando k-fold: {'algorithm': 'auto', 'metric': 'manhattan', 'n_neighbors': 15, 'p': 1, 'weights': 'uniform'}


In [18]:
# Predecir las etiquetas para los datos de prueba
resultados_df_knn = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_knn.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_knn_entrenamiento = pd.concat([resultados_df_knn, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_knn_entrenamiento["MODELO"]='KNeighbors'
resultados_df_knn_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_knn_entrenamiento


Precisión:  60.19
Exhaustividad:  64.56
Puntuación F1:  59.44
Exactitud:  64.56
Información Mutua Normalizada (NMI): 4.04
Índice Kappa de Cohen: 17.94


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,60.19,KNeighbors,Entrenamiento
1,Exhaustividad,64.56,KNeighbors,Entrenamiento
2,Puntuación F1,59.44,KNeighbors,Entrenamiento
3,Exactitud,64.56,KNeighbors,Entrenamiento
4,Información Mutua Normalizada (NMI),4.04,KNeighbors,Entrenamiento
5,Índice Kappa de Cohen,17.94,KNeighbors,Entrenamiento


In [19]:
# Predecir las etiquetas para los datos de prueba
resultados_df_knn = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_knn.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_knn_prueba = pd.concat([resultados_df_knn, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_knn_prueba["MODELO"]='KNeighbors'
resultados_df_knn_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_knn_prueba

Precisión:  51.65
Exhaustividad:  58.87
Puntuación F1:  53.6
Exactitud:  58.87
Información Mutua Normalizada (NMI): 0.09
Índice Kappa de Cohen: -0.73


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,51.65,KNeighbors,Prueba
1,Exhaustividad,58.87,KNeighbors,Prueba
2,Puntuación F1,53.60,KNeighbors,Prueba
3,Exactitud,58.87,KNeighbors,Prueba
4,Información Mutua Normalizada (NMI),0.09,KNeighbors,Prueba
5,Índice Kappa de Cohen,-0.73,KNeighbors,Prueba


In [20]:
resultados_df_knn = pd.concat([resultados_df_knn_prueba,resultados_df_knn_entrenamiento], ignore_index=True)
resultados_df_knn

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,51.65,KNeighbors,Prueba
1,Exhaustividad,58.87,KNeighbors,Prueba
2,Puntuación F1,53.60,KNeighbors,Prueba
3,Exactitud,58.87,KNeighbors,Prueba
4,Información Mutua Normalizada (NMI),0.09,KNeighbors,Prueba
5,Índice Kappa de Cohen,-0.73,KNeighbors,Prueba
6,Precisión,60.19,KNeighbors,Entrenamiento
7,Exhaustividad,64.56,KNeighbors,Entrenamiento
8,Puntuación F1,59.44,KNeighbors,Entrenamiento
9,Exactitud,64.56,KNeighbors,Entrenamiento


### SVC

In [21]:
def entrenar_modelo_svc_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 'kernel':  ['rbf'], 
            'C': [i/10000 for i in range(8,12,1)],
            'max_iter':[i for i in range(1,3,1)],
            'gamma' : [i/100 for i in range(90,110,5)],
            'random_state':[i for i in range(1,5,1)]}
    modelo = SVC()
    semilla=5
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = SVC(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_svc = entrenar_modelo_svc_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 56.805977710233016
Mejor valor PARAMETRO usando k-fold: {'C': 0.0008, 'gamma': 0.9, 'kernel': 'rbf', 'max_iter': 1, 'random_state': 1}


In [22]:
# Predecir las etiquetas para los datos de prueba
resultados_df_SVC = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_svc.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_svc_entrenamiento = pd.concat([resultados_df_SVC, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_svc_entrenamiento["MODELO"]='SVC'
resultados_df_svc_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_svc_entrenamiento

Precisión:  52.06
Exhaustividad:  59.43
Puntuación F1:  52.31
Exactitud:  59.43
Información Mutua Normalizada (NMI): 0.49
Índice Kappa de Cohen: 3.87


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,52.06,SVC,Entrenamiento
1,Exhaustividad,59.43,SVC,Entrenamiento
2,Puntuación F1,52.31,SVC,Entrenamiento
3,Exactitud,59.43,SVC,Entrenamiento
4,Información Mutua Normalizada (NMI),0.49,SVC,Entrenamiento
5,Índice Kappa de Cohen,3.87,SVC,Entrenamiento


In [23]:
# Predecir las etiquetas para los datos de prueba
resultados_df_SVC = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_svc.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_svc_prueba = pd.concat([resultados_df_SVC, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_svc_prueba["MODELO"]='SVC'
resultados_df_svc_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_svc_prueba

Precisión:  52.64
Exhaustividad:  60.7
Puntuación F1:  53.86
Exactitud:  60.7
Información Mutua Normalizada (NMI): 0.12
Índice Kappa de Cohen: 0.42


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,52.64,SVC,Prueba
1,Exhaustividad,60.70,SVC,Prueba
2,Puntuación F1,53.86,SVC,Prueba
3,Exactitud,60.70,SVC,Prueba
4,Información Mutua Normalizada (NMI),0.12,SVC,Prueba
5,Índice Kappa de Cohen,0.42,SVC,Prueba


In [24]:
resultados_df_svc = pd.concat([resultados_df_svc_prueba,resultados_df_svc_entrenamiento], ignore_index=True)
resultados_df_svc

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,52.64,SVC,Prueba
1,Exhaustividad,60.70,SVC,Prueba
2,Puntuación F1,53.86,SVC,Prueba
3,Exactitud,60.70,SVC,Prueba
4,Información Mutua Normalizada (NMI),0.12,SVC,Prueba
5,Índice Kappa de Cohen,0.42,SVC,Prueba
6,Precisión,52.06,SVC,Entrenamiento
7,Exhaustividad,59.43,SVC,Entrenamiento
8,Puntuación F1,52.31,SVC,Entrenamiento
9,Exactitud,59.43,SVC,Entrenamiento


### DECISION TREE

In [25]:
def entrenar_modelo_tree_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {          
            'max_depth':[i for i in range(3,6,1)],
            'min_samples_split' :  [i for i in range(1,3,1)],  
            'min_samples_leaf' : [i for i in range(1,3,1)], 
            'max_features' : [i for i in range(5,7,1)], 
            'splitter': ["best", "random"],
            'random_state': [i for i in range(1,4,1)],
            'criterion': ['entropy']}
    modelo = DecisionTreeClassifier()
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = DecisionTreeClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_tree = entrenar_modelo_tree_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 62.06585612968591
Mejor valor PARAMETRO usando k-fold: {'criterion': 'entropy', 'max_depth': 5, 'max_features': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'random_state': 1, 'splitter': 'random'}


In [26]:
# Predecir las etiquetas para los datos de prueba
resultados_df_tree = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_tree.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_tree_entrenamiento = pd.concat([resultados_df_tree, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_tree_entrenamiento["MODELO"]='DecisionTree'
resultados_df_tree_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_tree_entrenamiento

Precisión:  62.93
Exhaustividad:  63.2
Puntuación F1:  52.21
Exactitud:  63.2
Información Mutua Normalizada (NMI): 2.63
Índice Kappa de Cohen: 7.61


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,62.93,DecisionTree,Entrenamiento
1,Exhaustividad,63.20,DecisionTree,Entrenamiento
2,Puntuación F1,52.21,DecisionTree,Entrenamiento
3,Exactitud,63.20,DecisionTree,Entrenamiento
4,Información Mutua Normalizada (NMI),2.63,DecisionTree,Entrenamiento
5,Índice Kappa de Cohen,7.61,DecisionTree,Entrenamiento


In [27]:
# Predecir las etiquetas para los datos de prueba
resultados_df_tree = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_tree.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_tree_prueba = pd.concat([resultados_df_tree, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_tree_prueba["MODELO"]='DecisionTree'
resultados_df_tree_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_tree_prueba

Precisión:  55.6
Exhaustividad:  63.85
Puntuación F1:  53.1
Exactitud:  63.85
Información Mutua Normalizada (NMI): 0.23
Índice Kappa de Cohen: 2.33


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,55.60,DecisionTree,Prueba
1,Exhaustividad,63.85,DecisionTree,Prueba
2,Puntuación F1,53.10,DecisionTree,Prueba
3,Exactitud,63.85,DecisionTree,Prueba
4,Información Mutua Normalizada (NMI),0.23,DecisionTree,Prueba
5,Índice Kappa de Cohen,2.33,DecisionTree,Prueba


In [28]:
resultados_df_tree = pd.concat([resultados_df_tree_prueba,resultados_df_tree_entrenamiento], ignore_index=True)
resultados_df_tree

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,55.60,DecisionTree,Prueba
1,Exhaustividad,63.85,DecisionTree,Prueba
2,Puntuación F1,53.10,DecisionTree,Prueba
3,Exactitud,63.85,DecisionTree,Prueba
4,Información Mutua Normalizada (NMI),0.23,DecisionTree,Prueba
5,Índice Kappa de Cohen,2.33,DecisionTree,Prueba
6,Precisión,62.93,DecisionTree,Entrenamiento
7,Exhaustividad,63.20,DecisionTree,Entrenamiento
8,Puntuación F1,52.21,DecisionTree,Entrenamiento
9,Exactitud,63.20,DecisionTree,Entrenamiento


### NAIVE BAYES

In [29]:
def entrenar_modelo_gaussian_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {}
    modelo = GaussianNB()
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = GaussianNB(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_gaussian = entrenar_modelo_gaussian_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 60.993920972644375
Mejor valor PARAMETRO usando k-fold: {}


In [30]:
# Predecir las etiquetas para los datos de prueba
resultados_df_gaussian = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_gaussian.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_gaussian_entrenamiento = pd.concat([resultados_df_gaussian, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_gaussian_entrenamiento["MODELO"]='NaiveBayes'
resultados_df_gaussian_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_gaussian_entrenamiento

Precisión:  55.74
Exhaustividad:  61.57
Puntuación F1:  49.37
Exactitud:  61.57
Información Mutua Normalizada (NMI): 1.7
Índice Kappa de Cohen: 2.89


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,55.74,NaiveBayes,Entrenamiento
1,Exhaustividad,61.57,NaiveBayes,Entrenamiento
2,Puntuación F1,49.37,NaiveBayes,Entrenamiento
3,Exactitud,61.57,NaiveBayes,Entrenamiento
4,Información Mutua Normalizada (NMI),1.70,NaiveBayes,Entrenamiento
5,Índice Kappa de Cohen,2.89,NaiveBayes,Entrenamiento


In [31]:
# Predecir las etiquetas para los datos de prueba
resultados_df_gaussian = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_gaussian.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_gaussian_prueba = pd.concat([resultados_df_gaussian, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_gaussian_prueba["MODELO"]='NaiveBayes'
resultados_df_gaussian_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_gaussian_prueba

Precisión:  55.59
Exhaustividad:  64.01
Puntuación F1:  51.91
Exactitud:  64.01
Información Mutua Normalizada (NMI): 0.25
Índice Kappa de Cohen: 1.1


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,55.59,NaiveBayes,Prueba
1,Exhaustividad,64.01,NaiveBayes,Prueba
2,Puntuación F1,51.91,NaiveBayes,Prueba
3,Exactitud,64.01,NaiveBayes,Prueba
4,Información Mutua Normalizada (NMI),0.25,NaiveBayes,Prueba
5,Índice Kappa de Cohen,1.10,NaiveBayes,Prueba


In [32]:
resultados_df_gaussian = pd.concat([resultados_df_gaussian_prueba,resultados_df_gaussian_entrenamiento], ignore_index=True)
resultados_df_gaussian

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,55.59,NaiveBayes,Prueba
1,Exhaustividad,64.01,NaiveBayes,Prueba
2,Puntuación F1,51.91,NaiveBayes,Prueba
3,Exactitud,64.01,NaiveBayes,Prueba
4,Información Mutua Normalizada (NMI),0.25,NaiveBayes,Prueba
5,Índice Kappa de Cohen,1.10,NaiveBayes,Prueba
6,Precisión,55.74,NaiveBayes,Entrenamiento
7,Exhaustividad,61.57,NaiveBayes,Entrenamiento
8,Puntuación F1,49.37,NaiveBayes,Entrenamiento
9,Exactitud,61.57,NaiveBayes,Entrenamiento


### LDA

In [33]:
def entrenar_modelo_LDA_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 'solver':  ['svd','lsqr','eigen'],
            'n_components':[1,2,3,4,5,None],
            'shrinkage': ['auto',None, 0, 0.001, 0.01, 0.1, 0.5,1]}
    modelo = LinearDiscriminantAnalysis()
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = LinearDiscriminantAnalysis(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_LDA = entrenar_modelo_LDA_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 61.49392097264437
Mejor valor PARAMETRO usando k-fold: {'n_components': 1, 'shrinkage': None, 'solver': 'svd'}


In [34]:
# Predecir las etiquetas para los datos de prueba
resultados_df_LDA = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_LDA.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_LDA_entrenamiento = pd.concat([resultados_df_LDA, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_LDA_entrenamiento["MODELO"]='LDA'
resultados_df_LDA_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_LDA_entrenamiento

Precisión:  52.51
Exhaustividad:  61.21
Puntuación F1:  47.84
Exactitud:  61.21
Información Mutua Normalizada (NMI): 1.37
Índice Kappa de Cohen: 1.07


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,52.51,LDA,Entrenamiento
1,Exhaustividad,61.21,LDA,Entrenamiento
2,Puntuación F1,47.84,LDA,Entrenamiento
3,Exactitud,61.21,LDA,Entrenamiento
4,Información Mutua Normalizada (NMI),1.37,LDA,Entrenamiento
5,Índice Kappa de Cohen,1.07,LDA,Entrenamiento


In [35]:
# Predecir las etiquetas para los datos de prueba
resultados_df_LDA = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_LDA.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_LDA_prueba = pd.concat([resultados_df_LDA, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_LDA_prueba["MODELO"]='LDA'
resultados_df_LDA_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_LDA_prueba

Precisión:  51.19
Exhaustividad:  63.68
Puntuación F1:  50.93
Exactitud:  63.68
Información Mutua Normalizada (NMI): 0.13
Índice Kappa de Cohen: -0.39


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,51.19,LDA,Prueba
1,Exhaustividad,63.68,LDA,Prueba
2,Puntuación F1,50.93,LDA,Prueba
3,Exactitud,63.68,LDA,Prueba
4,Información Mutua Normalizada (NMI),0.13,LDA,Prueba
5,Índice Kappa de Cohen,-0.39,LDA,Prueba


In [36]:
resultados_df_LDA = pd.concat([resultados_df_LDA_prueba,resultados_df_LDA_entrenamiento], ignore_index=True)
resultados_df_LDA

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,51.19,LDA,Prueba
1,Exhaustividad,63.68,LDA,Prueba
2,Puntuación F1,50.93,LDA,Prueba
3,Exactitud,63.68,LDA,Prueba
4,Información Mutua Normalizada (NMI),0.13,LDA,Prueba
5,Índice Kappa de Cohen,-0.39,LDA,Prueba
6,Precisión,52.51,LDA,Entrenamiento
7,Exhaustividad,61.21,LDA,Entrenamiento
8,Puntuación F1,47.84,LDA,Entrenamiento
9,Exactitud,61.21,LDA,Entrenamiento


### BAGGINGCLASSIFIER

In [37]:
from sklearn.ensemble import BaggingClassifier
estimator = BaggingClassifier()
estimator.get_params()

{'bootstrap': True,
 'bootstrap_features': False,
 'estimator': None,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [38]:
def entrenar_modelo_BG_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'n_estimators': [i for i in range(750,760,5)],
            'max_samples' : [i/100.0 for i in range(40,50,5)],
            'max_features': [i/100.0 for i in range(60,65,5)],
            'bootstrap': [True], 
            'bootstrap_features': [True]}
    
    base_estimator= DecisionTreeClassifier(criterion= 'gini', 
                                max_depth=5, max_features= 3,min_samples_leaf= 4, 
                                min_samples_split = 8,random_state= 10, splitter= 'random')

    modelo = BaggingClassifier(estimator=base_estimator, oob_score=True, random_state=1)
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = BaggingClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo
X_trn = X_trn
Y_trn = Y_trn 
modelo_BG = entrenar_modelo_BG_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 61.495440729483285
Mejor valor PARAMETRO usando k-fold: {'bootstrap': True, 'bootstrap_features': True, 'max_features': 0.6, 'max_samples': 0.45, 'n_estimators': 755}


In [39]:
# Predecir las etiquetas para los datos de prueba
resultados_df_BG = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_BG.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_BG_entrenamiento = pd.concat([resultados_df_BG, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_BG_entrenamiento["MODELO"]='Bagging'
resultados_df_BG_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_BG_entrenamiento

Precisión:  88.53
Exhaustividad:  86.19
Puntuación F1:  84.57
Exactitud:  86.19
Información Mutua Normalizada (NMI): 51.83
Índice Kappa de Cohen: 69.88


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,88.53,Bagging,Entrenamiento
1,Exhaustividad,86.19,Bagging,Entrenamiento
2,Puntuación F1,84.57,Bagging,Entrenamiento
3,Exactitud,86.19,Bagging,Entrenamiento
4,Información Mutua Normalizada (NMI),51.83,Bagging,Entrenamiento
5,Índice Kappa de Cohen,69.88,Bagging,Entrenamiento


In [40]:
# Predecir las etiquetas para los datos de prueba
resultados_df_BG = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_BG.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_BG_prueba = pd.concat([resultados_df_BG, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_BG_prueba["MODELO"]='Bagging'
resultados_df_BG_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_BG_prueba

Precisión:  60.46
Exhaustividad:  65.17
Puntuación F1:  56.27
Exactitud:  65.17
Información Mutua Normalizada (NMI): 1.5
Índice Kappa de Cohen: 7.73


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,60.46,Bagging,Prueba
1,Exhaustividad,65.17,Bagging,Prueba
2,Puntuación F1,56.27,Bagging,Prueba
3,Exactitud,65.17,Bagging,Prueba
4,Información Mutua Normalizada (NMI),1.50,Bagging,Prueba
5,Índice Kappa de Cohen,7.73,Bagging,Prueba


In [41]:
resultados_df_BG = pd.concat([resultados_df_BG_prueba,resultados_df_BG_entrenamiento], ignore_index=True)
resultados_df_BG

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,60.46,Bagging,Prueba
1,Exhaustividad,65.17,Bagging,Prueba
2,Puntuación F1,56.27,Bagging,Prueba
3,Exactitud,65.17,Bagging,Prueba
4,Información Mutua Normalizada (NMI),1.50,Bagging,Prueba
5,Índice Kappa de Cohen,7.73,Bagging,Prueba
6,Precisión,88.53,Bagging,Entrenamiento
7,Exhaustividad,86.19,Bagging,Entrenamiento
8,Puntuación F1,84.57,Bagging,Entrenamiento
9,Exactitud,86.19,Bagging,Entrenamiento


### RANDOM FOREST

In [42]:
def entrenar_modelo_random_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 
                'min_samples_split' : [ 2 , 4 , 6 , 8 , 10 , 15 ],  
                'min_samples_leaf' : [ 1 , 3 , 5 , 7 , 9 ] 
                  
              }
    modelo = RandomForestClassifier()
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = RandomForestClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_random = entrenar_modelo_random_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 61.56332320162107
Mejor valor PARAMETRO usando k-fold: {'min_samples_leaf': 9, 'min_samples_split': 6}


In [43]:
# Predecir las etiquetas para los datos de prueba
resultados_df_random = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_random.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_random_entrenamiento = pd.concat([resultados_df_random, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_random_entrenamiento["MODELO"]='RandomForest'
resultados_df_random_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_random_entrenamiento

Precisión:  72.08
Exhaustividad:  69.89
Puntuación F1:  63.63
Exactitud:  69.89
Información Mutua Normalizada (NMI): 14.47
Índice Kappa de Cohen: 27.62


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,72.08,RandomForest,Entrenamiento
1,Exhaustividad,69.89,RandomForest,Entrenamiento
2,Puntuación F1,63.63,RandomForest,Entrenamiento
3,Exactitud,69.89,RandomForest,Entrenamiento
4,Información Mutua Normalizada (NMI),14.47,RandomForest,Entrenamiento
5,Índice Kappa de Cohen,27.62,RandomForest,Entrenamiento


In [44]:
# Predecir las etiquetas para los datos de prueba
resultados_df_random = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_random.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_random_prueba = pd.concat([resultados_df_random, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_random_prueba["MODELO"]='RandomForest'
resultados_df_random_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_random_prueba

Precisión:  54.34
Exhaustividad:  62.52
Puntuación F1:  54.4
Exactitud:  62.52
Información Mutua Normalizada (NMI): 0.16
Índice Kappa de Cohen: 2.72


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,54.34,RandomForest,Prueba
1,Exhaustividad,62.52,RandomForest,Prueba
2,Puntuación F1,54.40,RandomForest,Prueba
3,Exactitud,62.52,RandomForest,Prueba
4,Información Mutua Normalizada (NMI),0.16,RandomForest,Prueba
5,Índice Kappa de Cohen,2.72,RandomForest,Prueba


In [45]:
resultados_df_random = pd.concat([resultados_df_random_prueba,resultados_df_random_entrenamiento], ignore_index=True)
resultados_df_random

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,54.34,RandomForest,Prueba
1,Exhaustividad,62.52,RandomForest,Prueba
2,Puntuación F1,54.40,RandomForest,Prueba
3,Exactitud,62.52,RandomForest,Prueba
4,Información Mutua Normalizada (NMI),0.16,RandomForest,Prueba
5,Índice Kappa de Cohen,2.72,RandomForest,Prueba
6,Precisión,72.08,RandomForest,Entrenamiento
7,Exhaustividad,69.89,RandomForest,Entrenamiento
8,Puntuación F1,63.63,RandomForest,Entrenamiento
9,Exactitud,69.89,RandomForest,Entrenamiento


### EXTRA TREES CLASSIFIER

In [46]:
def entrenar_modelo_extra_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'min_samples_split' : [i for i in range(1,3,1)], 
                'min_samples_leaf' : [i for i in range(0,2,1)] }
    
    semilla=7            
    modelo = ExtraTreesClassifier(random_state=semilla, 
                                n_estimators=40, max_features=1,max_depth= 10,
                                min_samples_leaf=1,  min_samples_split = 2,
                                bootstrap=True,criterion='gini') 
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = ExtraTreesClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_extra = entrenar_modelo_extra_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 61.91995947315097
Mejor valor PARAMETRO usando k-fold: {'min_samples_leaf': 1, 'min_samples_split': 2}


In [47]:
# Predecir las etiquetas para los datos de prueba
resultados_df_extra = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_extra.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_extra_entrenamiento = pd.concat([resultados_df_extra, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_extra_entrenamiento["MODELO"]='ExtraTrees'
resultados_df_extra_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_extra_entrenamiento


Precisión:  99.65
Exhaustividad:  99.64
Puntuación F1:  99.64
Exactitud:  99.64
Información Mutua Normalizada (NMI): 97.31
Índice Kappa de Cohen: 99.3


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,99.65,ExtraTrees,Entrenamiento
1,Exhaustividad,99.64,ExtraTrees,Entrenamiento
2,Puntuación F1,99.64,ExtraTrees,Entrenamiento
3,Exactitud,99.64,ExtraTrees,Entrenamiento
4,Información Mutua Normalizada (NMI),97.31,ExtraTrees,Entrenamiento
5,Índice Kappa de Cohen,99.30,ExtraTrees,Entrenamiento


In [48]:
# Predecir las etiquetas para los datos de prueba
resultados_df_extra = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_extra.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_extra_prueba = pd.concat([resultados_df_extra, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_extra_prueba["MODELO"]='ExtraTrees'
resultados_df_extra_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_extra_prueba

Precisión:  54.09
Exhaustividad:  58.37
Puntuación F1:  55.66
Exactitud:  58.37
Información Mutua Normalizada (NMI): 0.31
Índice Kappa de Cohen: 3.83


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,54.09,ExtraTrees,Prueba
1,Exhaustividad,58.37,ExtraTrees,Prueba
2,Puntuación F1,55.66,ExtraTrees,Prueba
3,Exactitud,58.37,ExtraTrees,Prueba
4,Información Mutua Normalizada (NMI),0.31,ExtraTrees,Prueba
5,Índice Kappa de Cohen,3.83,ExtraTrees,Prueba


In [49]:
resultados_df_extra = pd.concat([resultados_df_extra_prueba,resultados_df_extra_entrenamiento], ignore_index=True)
resultados_df_extra

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,54.09,ExtraTrees,Prueba
1,Exhaustividad,58.37,ExtraTrees,Prueba
2,Puntuación F1,55.66,ExtraTrees,Prueba
3,Exactitud,58.37,ExtraTrees,Prueba
4,Información Mutua Normalizada (NMI),0.31,ExtraTrees,Prueba
5,Índice Kappa de Cohen,3.83,ExtraTrees,Prueba
6,Precisión,99.65,ExtraTrees,Entrenamiento
7,Exhaustividad,99.64,ExtraTrees,Entrenamiento
8,Puntuación F1,99.64,ExtraTrees,Entrenamiento
9,Exactitud,99.64,ExtraTrees,Entrenamiento


### ADABOOST

In [50]:
from sklearn.ensemble import AdaBoostClassifier            
estimator  = AdaBoostClassifier ()
estimator.get_params()

{'algorithm': 'SAMME.R',
 'estimator': None,
 'learning_rate': 1.0,
 'n_estimators': 50,
 'random_state': None}

In [51]:
def entrenar_modelo_ADA_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'learning_rate' : [i/10000.0 for i in range(5,20,5)]}
    semilla=7            
    modelo = AdaBoostClassifier(estimator = None,  algorithm = 'SAMME.R', 
                                random_state= None, n_estimators =50) 
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = AdaBoostClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_ADA = entrenar_modelo_ADA_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 61.28166160081052
Mejor valor PARAMETRO usando k-fold: {'learning_rate': 0.0015}


In [52]:
# Predecir las etiquetas para los datos de prueba
resultados_df_ADA = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_ADA.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_ADA_entrenamiento = pd.concat([resultados_df_ADA, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_ADA_entrenamiento["MODELO"]='AdaBoost'
resultados_df_ADA_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_ADA_entrenamiento

Precisión:  37.55
Exhaustividad:  61.28
Puntuación F1:  46.57
Exactitud:  61.28
Información Mutua Normalizada (NMI): 0.0
Índice Kappa de Cohen: 0.0


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,37.55,AdaBoost,Entrenamiento
1,Exhaustividad,61.28,AdaBoost,Entrenamiento
2,Puntuación F1,46.57,AdaBoost,Entrenamiento
3,Exactitud,61.28,AdaBoost,Entrenamiento
4,Información Mutua Normalizada (NMI),0.00,AdaBoost,Entrenamiento
5,Índice Kappa de Cohen,0.00,AdaBoost,Entrenamiento


In [53]:
# Predecir las etiquetas para los datos de prueba
resultados_df_ADA = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_ADA.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_ADA_prueba = pd.concat([resultados_df_ADA, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_ADA_prueba["MODELO"]='AdaBoost'
resultados_df_ADA_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_ADA_prueba

Precisión:  41.4
Exhaustividad:  64.34
Puntuación F1:  50.39
Exactitud:  64.34
Información Mutua Normalizada (NMI): 0.0
Índice Kappa de Cohen: 0.0


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,41.40,AdaBoost,Prueba
1,Exhaustividad,64.34,AdaBoost,Prueba
2,Puntuación F1,50.39,AdaBoost,Prueba
3,Exactitud,64.34,AdaBoost,Prueba
4,Información Mutua Normalizada (NMI),0.00,AdaBoost,Prueba
5,Índice Kappa de Cohen,0.00,AdaBoost,Prueba


In [54]:
resultados_df_ADA = pd.concat([resultados_df_ADA_prueba,resultados_df_ADA_entrenamiento], ignore_index=True)
resultados_df_ADA

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,41.40,AdaBoost,Prueba
1,Exhaustividad,64.34,AdaBoost,Prueba
2,Puntuación F1,50.39,AdaBoost,Prueba
3,Exactitud,64.34,AdaBoost,Prueba
4,Información Mutua Normalizada (NMI),0.00,AdaBoost,Prueba
5,Índice Kappa de Cohen,0.00,AdaBoost,Prueba
6,Precisión,37.55,AdaBoost,Entrenamiento
7,Exhaustividad,61.28,AdaBoost,Entrenamiento
8,Puntuación F1,46.57,AdaBoost,Entrenamiento
9,Exactitud,61.28,AdaBoost,Entrenamiento


### GRADIENT BOOSTING MACHINE

In [55]:
def entrenar_modelo_GD_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 
                'subsample' : [ 0.5,0.6,0.7 , 0.75 , 0.8 , 0.85 , 0.9 , 0.95 , 1 ]        
              }
    semilla=7
    modelo = GradientBoostingClassifier(random_state=semilla,
                                    n_estimators= 100,learning_rate= 0.1,max_depth= 2,
                                    min_samples_split= 2, min_samples_leaf= 3,max_features= 2)
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = GradientBoostingClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_GD = entrenar_modelo_GD_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 61.280648429584595
Mejor valor PARAMETRO usando k-fold: {'subsample': 0.9}


In [56]:
# Predecir las etiquetas para los datos de prueba
resultados_df_GD = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_GD.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_GD_entrenamiento = pd.concat([resultados_df_GD, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_GD_entrenamiento["MODELO"]='GradientBoosting'
resultados_df_GD_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_GD_entrenamiento

Precisión:  80.95
Exhaustividad:  75.37
Puntuación F1:  71.96
Exactitud:  75.37
Información Mutua Normalizada (NMI): 28.54
Índice Kappa de Cohen: 43.18


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.95,GradientBoosting,Entrenamiento
1,Exhaustividad,75.37,GradientBoosting,Entrenamiento
2,Puntuación F1,71.96,GradientBoosting,Entrenamiento
3,Exactitud,75.37,GradientBoosting,Entrenamiento
4,Información Mutua Normalizada (NMI),28.54,GradientBoosting,Entrenamiento
5,Índice Kappa de Cohen,43.18,GradientBoosting,Entrenamiento


In [57]:
# Predecir las etiquetas para los datos de prueba
resultados_df_GD = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_GD.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_GD_prueba = pd.concat([resultados_df_GD, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_GD_prueba["MODELO"]='GradientBoosting'
resultados_df_GD_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_GD_prueba

Precisión:  54.54
Exhaustividad:  61.53
Puntuación F1:  55.55
Exactitud:  61.53
Información Mutua Normalizada (NMI): 0.22
Índice Kappa de Cohen: 3.92


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,54.54,GradientBoosting,Prueba
1,Exhaustividad,61.53,GradientBoosting,Prueba
2,Puntuación F1,55.55,GradientBoosting,Prueba
3,Exactitud,61.53,GradientBoosting,Prueba
4,Información Mutua Normalizada (NMI),0.22,GradientBoosting,Prueba
5,Índice Kappa de Cohen,3.92,GradientBoosting,Prueba


In [58]:
resultados_df_GD = pd.concat([resultados_df_GD_prueba,resultados_df_GD_entrenamiento], ignore_index=True)
resultados_df_GD

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,54.54,GradientBoosting,Prueba
1,Exhaustividad,61.53,GradientBoosting,Prueba
2,Puntuación F1,55.55,GradientBoosting,Prueba
3,Exactitud,61.53,GradientBoosting,Prueba
4,Información Mutua Normalizada (NMI),0.22,GradientBoosting,Prueba
5,Índice Kappa de Cohen,3.92,GradientBoosting,Prueba
6,Precisión,80.95,GradientBoosting,Entrenamiento
7,Exhaustividad,75.37,GradientBoosting,Entrenamiento
8,Puntuación F1,71.96,GradientBoosting,Entrenamiento
9,Exactitud,75.37,GradientBoosting,Entrenamiento


### XGBOOST

In [59]:
def entrenar_modelo_XB_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'reg_alpha': [0,0.1,0.2,0.3,0.4,0.5],
                'reg_lambda':  [i/100.0 for i in range(130,150,5)]}
    semilla=7
    modelo = XGBClassifier(random_state=semilla,                       
                        n_estimators=40,colsample_bytree = 1, subsample =1,max_depth =2,
                        min_child_weight =6,gamma = 0.05,learning_rate = 0.3)
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = XGBClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_XB = entrenar_modelo_XB_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 62.06433637284701
Mejor valor PARAMETRO usando k-fold: {'reg_alpha': 0, 'reg_lambda': 1.45}


In [60]:
# Predecir las etiquetas para los datos de prueba
resultados_df_XB = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_XB.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_XB_entrenamiento = pd.concat([resultados_df_XB, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_XB_entrenamiento["MODELO"]=' XGB'
resultados_df_XB_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_XB_entrenamiento

Precisión:  98.13
Exhaustividad:  98.08
Puntuación F1:  98.07
Exactitud:  98.08
Información Mutua Normalizada (NMI): 89.29
Índice Kappa de Cohen: 96.17


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,98.13,XGB,Entrenamiento
1,Exhaustividad,98.08,XGB,Entrenamiento
2,Puntuación F1,98.07,XGB,Entrenamiento
3,Exactitud,98.08,XGB,Entrenamiento
4,Información Mutua Normalizada (NMI),89.29,XGB,Entrenamiento
5,Índice Kappa de Cohen,96.17,XGB,Entrenamiento


In [61]:
# Predecir las etiquetas para los datos de prueba
resultados_df_XB = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_XB.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_XB_prueba = pd.concat([resultados_df_XB, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_XB_prueba["MODELO"]='XGB'
resultados_df_XB_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_XB_prueba

Precisión:  53.56
Exhaustividad:  56.55
Puntuación F1:  54.87
Exactitud:  56.55
Información Mutua Normalizada (NMI): 0.14
Índice Kappa de Cohen: 3.0


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,53.56,XGB,Prueba
1,Exhaustividad,56.55,XGB,Prueba
2,Puntuación F1,54.87,XGB,Prueba
3,Exactitud,56.55,XGB,Prueba
4,Información Mutua Normalizada (NMI),0.14,XGB,Prueba
5,Índice Kappa de Cohen,3.00,XGB,Prueba


In [62]:
resultados_df_XB = pd.concat([resultados_df_XB_prueba,resultados_df_XB_entrenamiento], ignore_index=True)
resultados_df_XB

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,53.56,XGB,Prueba
1,Exhaustividad,56.55,XGB,Prueba
2,Puntuación F1,54.87,XGB,Prueba
3,Exactitud,56.55,XGB,Prueba
4,Información Mutua Normalizada (NMI),0.14,XGB,Prueba
5,Índice Kappa de Cohen,3.00,XGB,Prueba
6,Precisión,98.13,XGB,Entrenamiento
7,Exhaustividad,98.08,XGB,Entrenamiento
8,Puntuación F1,98.07,XGB,Entrenamiento
9,Exactitud,98.08,XGB,Entrenamiento


### CATBOOST

In [63]:
def entrenar_modelo_CB_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'border_count':[53],'l2_leaf_reg': [42]} 
    semilla=7
    modelo = CatBoostClassifier(random_state=semilla, verbose =0)
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = CatBoostClassifier(verbose=0,**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_CB = entrenar_modelo_CB_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 60.56838905775075
Mejor valor PARAMETRO usando k-fold: {'border_count': 53, 'l2_leaf_reg': 42}


In [64]:
# Predecir las etiquetas para los datos de prueba
resultados_df_CB = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_CB.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento.ravel())
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_CB_entrenamiento = pd.concat([resultados_df_CB, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_CB_entrenamiento["MODELO"]='CatBoost'
resultados_df_CB_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_CB_entrenamiento

Precisión:  71.61
Exhaustividad:  70.11
Puntuación F1:  64.1
Exactitud:  70.11
Información Mutua Normalizada (NMI): 14.24
Índice Kappa de Cohen: 28.47


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,71.61,CatBoost,Entrenamiento
1,Exhaustividad,70.11,CatBoost,Entrenamiento
2,Puntuación F1,64.10,CatBoost,Entrenamiento
3,Exactitud,70.11,CatBoost,Entrenamiento
4,Información Mutua Normalizada (NMI),14.24,CatBoost,Entrenamiento
5,Índice Kappa de Cohen,28.47,CatBoost,Entrenamiento


In [65]:
# Predecir las etiquetas para los datos de prueba
resultados_df_CB = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_CB.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba.ravel())
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_CB_prueba = pd.concat([resultados_df_CB, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_CB_prueba["MODELO"]='CatBoost'
resultados_df_CB_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_CB_prueba

Precisión:  54.1
Exhaustividad:  62.19
Puntuación F1:  54.77
Exactitud:  62.19
Información Mutua Normalizada (NMI): 0.49
Índice Kappa de Cohen: 3.18


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,54.10,CatBoost,Prueba
1,Exhaustividad,62.19,CatBoost,Prueba
2,Puntuación F1,54.77,CatBoost,Prueba
3,Exactitud,62.19,CatBoost,Prueba
4,Información Mutua Normalizada (NMI),0.49,CatBoost,Prueba
5,Índice Kappa de Cohen,3.18,CatBoost,Prueba


In [66]:
resultados_df_CB = pd.concat([resultados_df_CB_prueba,resultados_df_CB_entrenamiento], ignore_index=True)
resultados_df_CB

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,54.10,CatBoost,Prueba
1,Exhaustividad,62.19,CatBoost,Prueba
2,Puntuación F1,54.77,CatBoost,Prueba
3,Exactitud,62.19,CatBoost,Prueba
4,Información Mutua Normalizada (NMI),0.49,CatBoost,Prueba
5,Índice Kappa de Cohen,3.18,CatBoost,Prueba
6,Precisión,71.61,CatBoost,Entrenamiento
7,Exhaustividad,70.11,CatBoost,Entrenamiento
8,Puntuación F1,64.10,CatBoost,Entrenamiento
9,Exactitud,70.11,CatBoost,Entrenamiento


### LIGHT GBM

In [67]:
def entrenar_modelo_LIGHT_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = { 'min_child_samples' : [i for i in range(200, 10000, 100)]}
    semilla=7
    modelo = LGBMClassifier (random_state=semilla,                           
                        num_leaves =  10,max_depth = 1, n_estimators = 100,    
                        learning_rate = 0.1 ,class_weight=  None, subsample = 1,
                        colsample_bytree= 1, reg_alpha=  0, reg_lambda = 0,
                        min_split_gain = 0, boosting_type = 'gbdt')
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = LGBMClassifier(verbose=-1,**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_LIGHT = entrenar_modelo_LIGHT_con_transformacion(X_trn, Y_trn)

KeyboardInterrupt: 

In [ ]:
resultados_df_LIGHT = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_LIGHT.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_LIGHT_entrenamiento = pd.concat([resultados_df_LIGHT, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_LIGHT_entrenamiento["MODELO"]='LGBM'
resultados_df_LIGHT_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_LIGHT_entrenamiento

Precisión:  59.01
Exhaustividad:  62.35
Puntuación F1:  51.25
Exactitud:  62.35
Información Mutua Normalizada (NMI): 2.54
Índice Kappa de Cohen: 5.66


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,59.01,LGBM,Entrenamiento
1,Exhaustividad,62.35,LGBM,Entrenamiento
2,Puntuación F1,51.25,LGBM,Entrenamiento
3,Exactitud,62.35,LGBM,Entrenamiento
4,Información Mutua Normalizada (NMI),2.54,LGBM,Entrenamiento
5,Índice Kappa de Cohen,5.66,LGBM,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_LIGHT = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_LIGHT.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_LIGHT_prueba = pd.concat([resultados_df_LIGHT, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_LIGHT_prueba["MODELO"]='LGBM'
resultados_df_LIGHT_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_LIGHT_prueba

Precisión:  56.37
Exhaustividad:  64.01
Puntuación F1:  53.43
Exactitud:  64.01
Información Mutua Normalizada (NMI): 0.33
Índice Kappa de Cohen: 2.91


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,56.37,LGBM,Prueba
1,Exhaustividad,64.01,LGBM,Prueba
2,Puntuación F1,53.43,LGBM,Prueba
3,Exactitud,64.01,LGBM,Prueba
4,Información Mutua Normalizada (NMI),0.33,LGBM,Prueba
5,Índice Kappa de Cohen,2.91,LGBM,Prueba


In [ ]:
resultados_df_LIGHT = pd.concat([resultados_df_LIGHT_prueba,resultados_df_LIGHT_entrenamiento], ignore_index=True)
resultados_df_LIGHT

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,56.37,LGBM,Prueba
1,Exhaustividad,64.01,LGBM,Prueba
2,Puntuación F1,53.43,LGBM,Prueba
3,Exactitud,64.01,LGBM,Prueba
4,Información Mutua Normalizada (NMI),0.33,LGBM,Prueba
5,Índice Kappa de Cohen,2.91,LGBM,Prueba
6,Precisión,59.01,LGBM,Entrenamiento
7,Exhaustividad,62.35,LGBM,Entrenamiento
8,Puntuación F1,51.25,LGBM,Entrenamiento
9,Exactitud,62.35,LGBM,Entrenamiento


### VOTING - VOTACIÓN DURA

In [ ]:
def entrenar_modelo_voting_hard_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    
    modelo1 = GradientBoostingClassifier(random_state=semilla, n_estimators=800, 
                                    learning_rate = 0.01, max_depth = 2,
                                    max_features =2, min_samples_leaf = 9, 
                                    min_samples_split = 2, subsample = 1 )
    
    base_estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, 
                                    criterion='gini',
                                    max_depth=8, max_features=1, max_leaf_nodes=None,
                                    min_impurity_decrease=0.0,
                                    min_samples_leaf=1, min_samples_split=5,
                                    min_weight_fraction_leaf=0.0,
                                    random_state=5, splitter='random')

    modelo2 = AdaBoostClassifier(estimator=base_estimator,n_estimators=1200, 
                                    algorithm ='SAMME.R', learning_rate = 0.001, 
                                    random_state=semilla)

    modelo3 = ExtraTreesClassifier(n_estimators=300, max_features=None,
                                    bootstrap = False, max_depth = 11,min_samples_split = 4, 
                                    min_samples_leaf = 1)

    modelo4 = RandomForestClassifier (n_estimators=1100, max_features=None,
                                    min_samples_split = 5, max_depth= 3, 
                                    min_samples_leaf= 1)

    model = DecisionTreeClassifier(criterion= 'gini', 
                                    max_depth=6, 
                                    max_features= None, 
                                    min_samples_leaf= 9, 
                                    min_samples_split = 2, 
                                    random_state= 10, 
                                    splitter= 'random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, 
                                    random_state=semilla,
                                    bootstrap= True, bootstrap_features = False, 
                                    max_features = 0.9,
                                    max_samples= 0.95)

    modelo6 = DecisionTreeClassifier(criterion= 'gini', max_depth=6,max_features= None, 
                                    min_samples_leaf= 9, 
                                    min_samples_split = 2, random_state= 10, splitter= 'random')

    modelo7 = XGBClassifier(n_estimators= 40, random_state=semilla,colsample_bytree = 1, 
                                    subsample =1, reg_alpha = 0.2, reg_lambda= 1.35,
                                    learning_rate= 0.3,max_depth =2, min_child_weight =6,
                                    gamma = 0.05)
    metrica = 'accuracy'
    mejor_modelo = VotingClassifier(
    estimators=[('Gradient', modelo1), ('Adaboost', modelo2), 
                                    ('Extratrees', modelo3),('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6),
                                    ('XGB',modelo7)],voting='hard') 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, 
                                    cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_voting_hard = entrenar_modelo_voting_hard_con_transformacion(X_trn, Y_trn)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 61.634751773049636  % 


In [ ]:
resultados_df_voting_hard = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_voting_hard.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_voting_hard_entrenamiento = pd.concat([resultados_df_voting_hard, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_voting_hard_entrenamiento["MODELO"]='VotingHard'
resultados_df_voting_hard_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_voting_hard_entrenamiento

Precisión:  70.8
Exhaustividad:  65.55
Puntuación F1:  55.77
Exactitud:  65.55
Información Mutua Normalizada (NMI): 7.95
Índice Kappa de Cohen: 14.01


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,70.80,VotingHard,Entrenamiento
1,Exhaustividad,65.55,VotingHard,Entrenamiento
2,Puntuación F1,55.77,VotingHard,Entrenamiento
3,Exactitud,65.55,VotingHard,Entrenamiento
4,Información Mutua Normalizada (NMI),7.95,VotingHard,Entrenamiento
5,Índice Kappa de Cohen,14.01,VotingHard,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_voting_hard = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_voting_hard.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_voting_hard_prueba = pd.concat([resultados_df_voting_hard, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_voting_hard_prueba["MODELO"]='VotingHard'
resultados_df_voting_hard_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_voting_hard_prueba

Precisión:  56.68
Exhaustividad:  64.18
Puntuación F1:  53.05
Exactitud:  64.18
Información Mutua Normalizada (NMI): 0.36
Índice Kappa de Cohen: 2.67


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,56.68,VotingHard,Prueba
1,Exhaustividad,64.18,VotingHard,Prueba
2,Puntuación F1,53.05,VotingHard,Prueba
3,Exactitud,64.18,VotingHard,Prueba
4,Información Mutua Normalizada (NMI),0.36,VotingHard,Prueba
5,Índice Kappa de Cohen,2.67,VotingHard,Prueba


In [ ]:
resultados_df_voting_hard = pd.concat([resultados_df_voting_hard_prueba,resultados_df_voting_hard_entrenamiento], ignore_index=True)
resultados_df_voting_hard

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,56.68,VotingHard,Prueba
1,Exhaustividad,64.18,VotingHard,Prueba
2,Puntuación F1,53.05,VotingHard,Prueba
3,Exactitud,64.18,VotingHard,Prueba
4,Información Mutua Normalizada (NMI),0.36,VotingHard,Prueba
5,Índice Kappa de Cohen,2.67,VotingHard,Prueba
6,Precisión,70.80,VotingHard,Entrenamiento
7,Exhaustividad,65.55,VotingHard,Entrenamiento
8,Puntuación F1,55.77,VotingHard,Entrenamiento
9,Exactitud,65.55,VotingHard,Entrenamiento


### VOTING - VOTACIÓN SUAVE

In [ ]:
def entrenar_modelo_voting_soft_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = GradientBoostingClassifier(random_state=semilla, n_estimators=800, 
                                    learning_rate = 0.01, max_depth = 2,
                                    max_features =2, min_samples_leaf = 9, 
                                    min_samples_split = 2, subsample = 1 )
    
    base_estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                                    max_depth=8, max_features=1, max_leaf_nodes=None,
                                    min_impurity_decrease=0.0,
                                    min_samples_leaf=1, min_samples_split=5,
                                    min_weight_fraction_leaf=0.0,
                                    random_state=5, splitter='random')

    modelo2 = AdaBoostClassifier(estimator=base_estimator,n_estimators=1200, 
                                    algorithm ='SAMME.R', learning_rate = 0.001, 
                                    random_state=semilla)

    modelo3 = ExtraTreesClassifier(n_estimators=300, max_features=None,bootstrap = False, 
                                    max_depth = 11,min_samples_split = 4, 
                                    min_samples_leaf = 1)

    modelo4 = RandomForestClassifier (n_estimators=1100, max_features=None,
                                    min_samples_split = 5, max_depth= 3, min_samples_leaf= 1)

    model = DecisionTreeClassifier(criterion= 'gini', 
                                    max_depth=6, 
                                    max_features= None, 
                                    min_samples_leaf= 9, 
                                    min_samples_split = 2, 
                                    random_state= 10, 
                                    splitter= 'random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, random_state=semilla,
                                    bootstrap= True, bootstrap_features = False, 
                                    max_features = 0.9, max_samples= 0.95)

    modelo6 = DecisionTreeClassifier(criterion= 'gini', max_depth=6,max_features= None, 
                                    min_samples_leaf= 9, min_samples_split = 2, 
                                    random_state= 10, splitter= 'random')
    metrica = 'accuracy'
    mejor_modelo = VotingClassifier(
    estimators=[('Gradient', modelo1), ('Adaboost', modelo2), ('Extratrees', modelo3),
                                    ('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6)],
    voting='soft',weights=[0.9,0.7,0.9,0.9,0.9,0.9]) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_voting_soft = entrenar_modelo_voting_soft_con_transformacion(X_trn, Y_trn)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 62.13323201621074  % 


In [ ]:
resultados_df_voting_soft = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_voting_soft.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_voting_soft_entrenamiento = pd.concat([resultados_df_voting_soft, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_voting_soft_entrenamiento["MODELO"]='VotingSoft'
resultados_df_voting_soft_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_voting_soft_entrenamiento

Precisión:  73.49
Exhaustividad:  67.76
Puntuación F1:  59.44
Exactitud:  67.76
Información Mutua Normalizada (NMI): 14.05
Índice Kappa de Cohen: 20.53


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,73.49,VotingSoft,Entrenamiento
1,Exhaustividad,67.76,VotingSoft,Entrenamiento
2,Puntuación F1,59.44,VotingSoft,Entrenamiento
3,Exactitud,67.76,VotingSoft,Entrenamiento
4,Información Mutua Normalizada (NMI),14.05,VotingSoft,Entrenamiento
5,Índice Kappa de Cohen,20.53,VotingSoft,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_voting_soft = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_voting_soft.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_voting_soft_prueba = pd.concat([resultados_df_voting_soft, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_voting_soft_prueba["MODELO"]='VotingSoft'
resultados_df_voting_soft_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_voting_soft_prueba

Precisión:  57.21
Exhaustividad:  64.34
Puntuación F1:  53.89
Exactitud:  64.34
Información Mutua Normalizada (NMI): 0.79
Índice Kappa de Cohen: 3.94


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,57.21,VotingSoft,Prueba
1,Exhaustividad,64.34,VotingSoft,Prueba
2,Puntuación F1,53.89,VotingSoft,Prueba
3,Exactitud,64.34,VotingSoft,Prueba
4,Información Mutua Normalizada (NMI),0.79,VotingSoft,Prueba
5,Índice Kappa de Cohen,3.94,VotingSoft,Prueba


In [ ]:
resultados_df_voting_soft = pd.concat([resultados_df_voting_soft_prueba,resultados_df_voting_soft_entrenamiento], ignore_index=True)
resultados_df_voting_soft

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,57.21,VotingSoft,Prueba
1,Exhaustividad,64.34,VotingSoft,Prueba
2,Puntuación F1,53.89,VotingSoft,Prueba
3,Exactitud,64.34,VotingSoft,Prueba
4,Información Mutua Normalizada (NMI),0.79,VotingSoft,Prueba
5,Índice Kappa de Cohen,3.94,VotingSoft,Prueba
6,Precisión,73.49,VotingSoft,Entrenamiento
7,Exhaustividad,67.76,VotingSoft,Entrenamiento
8,Puntuación F1,59.44,VotingSoft,Entrenamiento
9,Exactitud,67.76,VotingSoft,Entrenamiento


### STACKING ( METAMODELO LINEAL)

In [ ]:
def entrenar_modelo_stacking_lineal_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    base_estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                                    max_depth=8, max_features=1, max_leaf_nodes=None,
                                    min_impurity_decrease=0.0,
                                    min_samples_leaf=1, min_samples_split=5,
                                    min_weight_fraction_leaf=0.0,
                                    random_state=5, splitter='random')
    modelo2 = AdaBoostClassifier(estimator=base_estimator,n_estimators=500, 
                                    algorithm ='SAMME.R', learning_rate = 0.001, 
                                    random_state=semilla)

    modelo3 = ExtraTreesClassifier(n_estimators=300, max_features=None,bootstrap = False, 
                                    max_depth = 11,min_samples_split = 4, 
                                    min_samples_leaf = 1)

    modelo4 = RandomForestClassifier (n_estimators=300, max_features=None,
                                    min_samples_split = 5, max_depth= 3, min_samples_leaf= 1)
    model = DecisionTreeClassifier(criterion= 'gini', 
                                    max_depth=6, 
                                    max_features= None, 
                                    min_samples_leaf= 9, 
                                    min_samples_split = 2, 
                                    random_state= 10, 
                                    splitter= 'random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, random_state=semilla,
                                    bootstrap= True, bootstrap_features = False, 
                                    max_features = 0.9,
                                    max_samples= 0.95)

    modelo6 = DecisionTreeClassifier(criterion= 'gini', max_depth=6,max_features= None, 
                                    min_samples_leaf= 9, 
                                    min_samples_split = 2, random_state= 10, splitter= 'random')
    estimador_final = LogisticRegression()
    metrica = 'accuracy'
    mejor_modelo = StackingClassifier(
    estimators=[ ('Adaboost', modelo2), ('Extratrees', modelo3),('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6)], 
                                    final_estimator=estimador_final) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_stacking_lineal = entrenar_modelo_stacking_lineal_con_transformacion(X_trn, Y_trn)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 62.134245187436676  % 


In [ ]:
resultados_df_stacking_lineal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_stacking_lineal.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_stacking_lineal_entrenamiento = pd.concat([resultados_df_stacking_lineal, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_stacking_lineal_entrenamiento["MODELO"]='StackingLineal'
resultados_df_stacking_lineal_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_stacking_lineal_entrenamiento

Precisión:  76.09
Exhaustividad:  72.17
Puntuación F1:  66.43
Exactitud:  72.17
Información Mutua Normalizada (NMI): 22.62
Índice Kappa de Cohen: 33.26


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,76.09,StackingLineal,Entrenamiento
1,Exhaustividad,72.17,StackingLineal,Entrenamiento
2,Puntuación F1,66.43,StackingLineal,Entrenamiento
3,Exactitud,72.17,StackingLineal,Entrenamiento
4,Información Mutua Normalizada (NMI),22.62,StackingLineal,Entrenamiento
5,Índice Kappa de Cohen,33.26,StackingLineal,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_stacking_lineal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_stacking_lineal.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_stacking_lineal_prueba = pd.concat([resultados_df_stacking_lineal, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_stacking_lineal_prueba["MODELO"]='StackingLineal'
resultados_df_stacking_lineal_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_stacking_lineal_prueba

Precisión:  60.31
Exhaustividad:  65.34
Puntuación F1:  55.84
Exactitud:  65.34
Información Mutua Normalizada (NMI): 2.1
Índice Kappa de Cohen: 7.53


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,60.31,StackingLineal,Prueba
1,Exhaustividad,65.34,StackingLineal,Prueba
2,Puntuación F1,55.84,StackingLineal,Prueba
3,Exactitud,65.34,StackingLineal,Prueba
4,Información Mutua Normalizada (NMI),2.10,StackingLineal,Prueba
5,Índice Kappa de Cohen,7.53,StackingLineal,Prueba


In [ ]:
resultados_df_stacking_lineal = pd.concat([resultados_df_stacking_lineal_prueba,resultados_df_stacking_lineal_entrenamiento], ignore_index=True)
resultados_df_stacking_lineal

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,60.31,StackingLineal,Prueba
1,Exhaustividad,65.34,StackingLineal,Prueba
2,Puntuación F1,55.84,StackingLineal,Prueba
3,Exactitud,65.34,StackingLineal,Prueba
4,Información Mutua Normalizada (NMI),2.10,StackingLineal,Prueba
5,Índice Kappa de Cohen,7.53,StackingLineal,Prueba
6,Precisión,76.09,StackingLineal,Entrenamiento
7,Exhaustividad,72.17,StackingLineal,Entrenamiento
8,Puntuación F1,66.43,StackingLineal,Entrenamiento
9,Exactitud,72.17,StackingLineal,Entrenamiento


### STACKING (METAMODELO NO LINEAL)

In [ ]:
def entrenar_modelo_stacking_nolineal_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo3 = ExtraTreesClassifier(n_estimators=400, max_features=None,bootstrap = False, 
                                    max_depth = 11,min_samples_split = 4, 
                                    min_samples_leaf = 1)

    modelo4 = RandomForestClassifier (n_estimators=500, max_features=None,
                                    min_samples_split = 5, max_depth= 3, min_samples_leaf= 1)
    model = DecisionTreeClassifier(criterion= 'gini', 
                                    max_depth=6, max_features= None,  min_samples_leaf= 9, 
                                    min_samples_split = 2, 
                                    random_state= 10, splitter= 'random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, random_state=semilla,
                                    bootstrap= True, bootstrap_features = False, 
                                    max_features = 0.9, max_samples= 0.95)

    modelo6 = DecisionTreeClassifier(criterion= 'gini', max_depth=6,max_features= None, 
                                    min_samples_leaf= 9, min_samples_split = 2, 
                                    random_state= 10, splitter= 'random')
    estimador_final = ExtraTreesClassifier(n_estimators=100, max_features=None,
                                    bootstrap = False, max_depth = 11,min_samples_split = 4, 
                                    min_samples_leaf = 1)
    metrica = 'accuracy'
    mejor_modelo = StackingClassifier(
    estimators=[  ('Extratrees', modelo3),('Random Forest',modelo4),('Bagging',modelo5),
                                    ('Decision tree',modelo6)], 
                                    final_estimator=estimador_final) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_stacking_nolineal = entrenar_modelo_stacking_nolineal_con_transformacion(X_trn, Y_trn)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 61.636778115501514  % 


In [ ]:
resultados_df_stacking_nolineal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_stacking_nolineal.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_stacking_nolineal_entrenamiento = pd.concat([resultados_df_stacking_nolineal, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_stacking_nolineal_entrenamiento["MODELO"]='Stackingnolineal'
resultados_df_stacking_nolineal_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_stacking_nolineal_entrenamiento

Precisión:  72.37
Exhaustividad:  71.89
Puntuación F1:  66.63
Exactitud:  71.89
Información Mutua Normalizada (NMI): 17.19
Índice Kappa de Cohen: 33.66


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,72.37,Stackingnolineal,Entrenamiento
1,Exhaustividad,71.89,Stackingnolineal,Entrenamiento
2,Puntuación F1,66.63,Stackingnolineal,Entrenamiento
3,Exactitud,71.89,Stackingnolineal,Entrenamiento
4,Información Mutua Normalizada (NMI),17.19,Stackingnolineal,Entrenamiento
5,Índice Kappa de Cohen,33.66,Stackingnolineal,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_stacking_nolineal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_stacking_nolineal.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_stacking_nolineal_prueba = pd.concat([resultados_df_stacking_nolineal, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_stacking_nolineal_prueba["MODELO"]='Stackingnolineal'
resultados_df_stacking_nolineal_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_stacking_nolineal_prueba

Precisión:  57.12
Exhaustividad:  64.01
Puntuación F1:  55.76
Exactitud:  64.01
Información Mutua Normalizada (NMI): 0.95
Índice Kappa de Cohen: 6.09


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,57.12,Stackingnolineal,Prueba
1,Exhaustividad,64.01,Stackingnolineal,Prueba
2,Puntuación F1,55.76,Stackingnolineal,Prueba
3,Exactitud,64.01,Stackingnolineal,Prueba
4,Información Mutua Normalizada (NMI),0.95,Stackingnolineal,Prueba
5,Índice Kappa de Cohen,6.09,Stackingnolineal,Prueba


In [ ]:
resultados_df_stacking_nolineal = pd.concat([resultados_df_stacking_nolineal_prueba,resultados_df_stacking_nolineal_entrenamiento], ignore_index=True)
resultados_df_stacking_nolineal

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,57.12,Stackingnolineal,Prueba
1,Exhaustividad,64.01,Stackingnolineal,Prueba
2,Puntuación F1,55.76,Stackingnolineal,Prueba
3,Exactitud,64.01,Stackingnolineal,Prueba
4,Información Mutua Normalizada (NMI),0.95,Stackingnolineal,Prueba
5,Índice Kappa de Cohen,6.09,Stackingnolineal,Prueba
6,Precisión,72.37,Stackingnolineal,Entrenamiento
7,Exhaustividad,71.89,Stackingnolineal,Entrenamiento
8,Puntuación F1,66.63,Stackingnolineal,Entrenamiento
9,Exactitud,71.89,Stackingnolineal,Entrenamiento


### VOTING WEIGHTED VOTING 

In [ ]:
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

def weighted_voting_ensemble(X_trn, Y_trn):
    X_trn_transformado = X_trn
    modelos = [
        GradientBoostingClassifier(random_state=7, n_estimators=800, learning_rate=0.01, max_depth=2, max_features=2, min_samples_leaf=9, min_samples_split=2, subsample=1),
        AdaBoostClassifier(estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini', max_depth=8, max_features=1, max_leaf_nodes=None, min_impurity_decrease=0.0, min_samples_leaf=1, min_samples_split=5, min_weight_fraction_leaf=0.0, random_state=5, splitter='random'), n_estimators=1200, algorithm='SAMME.R', learning_rate=0.001, random_state=7),
        ExtraTreesClassifier(n_estimators=300, max_features=None, bootstrap=False, max_depth=11, min_samples_split=4, min_samples_leaf=1),
        RandomForestClassifier(n_estimators=300, max_features=None, min_samples_split=5, max_depth=3, min_samples_leaf=1),
        BaggingClassifier(estimator=DecisionTreeClassifier(criterion='gini', max_depth=6, max_features=None, min_samples_leaf=9, min_samples_split=2, random_state=10, splitter='random'), n_estimators=50, random_state=7, bootstrap=True, bootstrap_features=False, max_features=0.9, max_samples=0.95),
        DecisionTreeClassifier(criterion='gini', max_depth=6, max_features=None, min_samples_leaf=9, min_samples_split=2, random_state=10, splitter='random'),
        XGBClassifier(n_estimators=40, random_state=7, colsample_bytree=1, subsample=1, reg_alpha=0.2, reg_lambda=1.35, learning_rate=0.3, max_depth=2, min_child_weight=6, gamma=0.05)
    ]
    
    # Inicializar pesos para el voto ponderado
    pesos = np.ones(len(modelos))
    
    # Entrenar cada modelo y obtener su precisión
    precisión_modelos = []
    for modelo in modelos:
        modelo.fit(X_trn_transformado, Y_trn)  # <-- Ajustar el modelo
        precisión = np.mean(cross_val_score(modelo, X_trn_transformado, Y_trn, cv=KFold(n_splits=10, random_state=7, shuffle=True), scoring='accuracy'))
        precisión_modelos.append(precisión)
    
    # Calcular los pesos como inversa de la precisión de cada modelo
    suma_precisión = sum(precisión_modelos)
    for i in range(len(precisión_modelos)):
        pesos[i] = suma_precisión / (precisión_modelos[i] * len(modelos))
    
    # Devolver modelos y pesos
    return modelos, pesos

def weighted_voting_predict(modelos, pesos, X_test):
    # Obtener las predicciones de cada modelo
    predicciones_modelos = [modelo.predict(X_test) for modelo in modelos]
    
    # Convertir los pesos a float64
    pesos_float64 = np.array(pesos, dtype=np.float64)
    
    # Calcular las predicciones ponderadas
    predicciones_ponderadas = np.zeros_like(predicciones_modelos[0], dtype=np.float64)
    for i, predicciones_modelo in enumerate(predicciones_modelos):
        predicciones_ponderadas += predicciones_modelo * pesos_float64[i]
    
    # Normalizar las predicciones ponderadas
    predicciones_ponderadas /= len(modelos)
    predicciones_ponderadas = np.round(predicciones_ponderadas).astype(int)
    
    return predicciones_ponderadas

# Utilizar la función
modelos, pesos = weighted_voting_ensemble(X_trn, Y_trn)

# Predecir las etiquetas para los datos de prueba
Y_pred = weighted_voting_predict(modelos, pesos, X_tst)

# Calcular la precisión de las predicciones
precision = accuracy_score(Y_tst, Y_pred)
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)
print("Precisión del modelo (Ensamblaje con voto ponderado): {:.2f}%".format(precision * 100))


Precisión:  0.6417910447761194
Exhaustividad (Recall):  0.6417910447761194
Puntuación F1:  0.5305327507628695
Exactitud:  0.6417910447761194
Precisión del modelo (Ensamblaje con voto ponderado): 64.18%


In [ ]:
resultados_df_weighted_voting = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= weighted_voting_predict(modelos, pesos, X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_weighted_voting_entrenamiento = pd.concat([resultados_df_weighted_voting, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_weighted_voting_entrenamiento["MODELO"]='VotingWeighted'
resultados_df_weighted_voting_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_weighted_voting_entrenamiento

Precisión:  71.31
Exhaustividad:  65.62
Puntuación F1:  55.84
Exactitud:  65.62
Información Mutua Normalizada (NMI): 9.43
Índice Kappa de Cohen: 14.23


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,71.31,VotingWeighted,Entrenamiento
1,Exhaustividad,65.62,VotingWeighted,Entrenamiento
2,Puntuación F1,55.84,VotingWeighted,Entrenamiento
3,Exactitud,65.62,VotingWeighted,Entrenamiento
4,Información Mutua Normalizada (NMI),9.43,VotingWeighted,Entrenamiento
5,Índice Kappa de Cohen,14.23,VotingWeighted,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_weighted_voting = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba= weighted_voting_predict(modelos, pesos, X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_weighted_voting_prueba = pd.concat([resultados_df_weighted_voting, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_weighted_voting_prueba["MODELO"]='VotingWeighted'
resultados_df_weighted_voting_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_weighted_voting_prueba

Precisión:  56.68
Exhaustividad:  64.18
Puntuación F1:  53.05
Exactitud:  64.18
Información Mutua Normalizada (NMI): 0.36
Índice Kappa de Cohen: 2.67


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,56.68,VotingWeighted,Prueba
1,Exhaustividad,64.18,VotingWeighted,Prueba
2,Puntuación F1,53.05,VotingWeighted,Prueba
3,Exactitud,64.18,VotingWeighted,Prueba
4,Información Mutua Normalizada (NMI),0.36,VotingWeighted,Prueba
5,Índice Kappa de Cohen,2.67,VotingWeighted,Prueba


In [ ]:
resultados_df_weighted_voting = pd.concat([resultados_df_weighted_voting_prueba,resultados_df_weighted_voting_entrenamiento], ignore_index=True)
resultados_df_weighted_voting

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,56.68,VotingWeighted,Prueba
1,Exhaustividad,64.18,VotingWeighted,Prueba
2,Puntuación F1,53.05,VotingWeighted,Prueba
3,Exactitud,64.18,VotingWeighted,Prueba
4,Información Mutua Normalizada (NMI),0.36,VotingWeighted,Prueba
5,Índice Kappa de Cohen,2.67,VotingWeighted,Prueba
6,Precisión,71.31,VotingWeighted,Entrenamiento
7,Exhaustividad,65.62,VotingWeighted,Entrenamiento
8,Puntuación F1,55.84,VotingWeighted,Entrenamiento
9,Exactitud,65.62,VotingWeighted,Entrenamiento


### SUPER APRENDIZ MLENS

In [ ]:
Y_trn = Y_trn.astype(int)
def entrenar_modelo_super_aprendiz(X_trn, Y_trn):
    X_trn_transformado = X_trn
    semilla = 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo3 = ExtraTreesClassifier(n_estimators=400, max_features=None, bootstrap=False, 
                                   max_depth=11, min_samples_split=4, min_samples_leaf=1)

    modelo4 = RandomForestClassifier(n_estimators=500, max_features=None,
                                     min_samples_split=5, max_depth=3, min_samples_leaf=1)
    model = DecisionTreeClassifier(criterion='gini', 
                                   max_depth=6, max_features=None,  min_samples_leaf=9, 
                                   min_samples_split=2, 
                                   random_state=10, splitter='random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, random_state=semilla,
                                bootstrap=True, bootstrap_features=False, 
                                max_features=0.9, max_samples=0.95)

    modelo6 = DecisionTreeClassifier(criterion='gini', max_depth=6, max_features=None, 
                                     min_samples_leaf=9, min_samples_split=2, 
                                     random_state=10, splitter='random')
    
    modelo7 = GradientBoostingClassifier(n_estimators=50, random_state=semilla,
                                     max_depth= 2,
                                    min_samples_split= 13,
                                    min_samples_leaf= 1,
                                    subsample =1,
                                    learning_rate= 0.06                                    
                                    ) 

    estimadores = [('Extratrees', modelo3), ('Random Forest', modelo4), 
                   ('Bagging', modelo5), ('Decision tree', modelo6),('Gradient',modelo7)]
    
    super_learner = SuperLearner(folds=10, random_state=semilla, verbose=2)
    super_learner.add(estimadores)
    
    estimador_final = ExtraTreesClassifier(n_estimators=100, max_features=None,
                                           bootstrap=False, max_depth=11, min_samples_split=4, 
                                           min_samples_leaf=1)
    
    super_learner.add_meta(estimador_final)
    
    super_learner.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(super_learner, X_trn_transformado, Y_trn, cv=kfold, scoring='accuracy')
    
    print("Rendimiento del modelo:") 
    print("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean() * 100, "%") 
    return super_learner


X_trn = X_trn
Y_trn = Y_trn 

modelo_superaprendiz = entrenar_modelo_super_aprendiz(X_trn, Y_trn)



Fitting 2 layers


Processing layer-1             done | 00:00:13
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:13

Fitting 2 layers
Processing layer-1             done | 00:00:13
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:13

Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00

Fitting 2 layers
Processing layer-1             done | 00:00:13
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:14

Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00

Fitting 2 layers
Processing layer-1             done | 00:00:13
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:14

Predicting 2 layers
Processing layer-1             done |

In [ ]:
print(modelo_superaprendiz.data)

                          ft-m  ft-s  pt-m  pt-s
layer-1  bagging          2.06  0.12  0.16  0.03
layer-1  decision-tree    0.03  0.01  0.01  0.02
layer-1  extratrees       6.73  0.21  0.84  0.20
layer-1  gradient         3.79  0.42  0.02  0.03
layer-1  random-forest    6.29  0.57  0.06  0.04



In [ ]:
resultados_df_superaprendiz = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_superaprendiz.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_superaprendiz_entrenamiento = pd.concat([resultados_df_superaprendiz, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_superaprendiz_entrenamiento["MODELO"]='SuperAprendiz'
resultados_df_superaprendiz_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_superaprendiz_entrenamiento


Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00
Precisión:  84.16
Exhaustividad:  85.14
Puntuación F1:  82.66
Exactitud:  85.14
Información Mutua Normalizada (NMI): 54.33
Índice Kappa de Cohen: 69.72


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,84.16,SuperAprendiz,Entrenamiento
1,Exhaustividad,85.14,SuperAprendiz,Entrenamiento
2,Puntuación F1,82.66,SuperAprendiz,Entrenamiento
3,Exactitud,85.14,SuperAprendiz,Entrenamiento
4,Información Mutua Normalizada (NMI),54.33,SuperAprendiz,Entrenamiento
5,Índice Kappa de Cohen,69.72,SuperAprendiz,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_superaprendiz = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_superaprendiz.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_superaprendiz_prueba = pd.concat([resultados_df_superaprendiz, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_superaprendiz_prueba["MODELO"]='SuperAprendiz'
resultados_df_superaprendiz_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_superaprendiz_prueba


Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00
Precisión:  77.1
Exhaustividad:  78.41
Puntuación F1:  76.62
Exactitud:  78.41
Información Mutua Normalizada (NMI): 32.28
Índice Kappa de Cohen: 55.91


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,77.10,SuperAprendiz,Prueba
1,Exhaustividad,78.41,SuperAprendiz,Prueba
2,Puntuación F1,76.62,SuperAprendiz,Prueba
3,Exactitud,78.41,SuperAprendiz,Prueba
4,Información Mutua Normalizada (NMI),32.28,SuperAprendiz,Prueba
5,Índice Kappa de Cohen,55.91,SuperAprendiz,Prueba


In [ ]:
resultados_df_superaprendiz = pd.concat([resultados_df_superaprendiz_prueba,resultados_df_superaprendiz_entrenamiento], ignore_index=True)
resultados_df_superaprendiz

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,77.10,SuperAprendiz,Prueba
1,Exhaustividad,78.41,SuperAprendiz,Prueba
2,Puntuación F1,76.62,SuperAprendiz,Prueba
3,Exactitud,78.41,SuperAprendiz,Prueba
4,Información Mutua Normalizada (NMI),32.28,SuperAprendiz,Prueba
5,Índice Kappa de Cohen,55.91,SuperAprendiz,Prueba
6,Precisión,84.16,SuperAprendiz,Entrenamiento
7,Exhaustividad,85.14,SuperAprendiz,Entrenamiento
8,Puntuación F1,82.66,SuperAprendiz,Entrenamiento
9,Exactitud,85.14,SuperAprendiz,Entrenamiento


### SUPER APRENDIZ MULTICAPA

In [ ]:
Y_trn = Y_trn.astype(int)
def entrenar_modelo_super_aprendiz_dos_capas(X_trn, Y_trn):
    X_trn_transformado = X_trn
    semilla = 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo3 = ExtraTreesClassifier(n_estimators=400, max_features=None, bootstrap=False, 
                                   max_depth=11, min_samples_split=4, min_samples_leaf=1)

    modelo4 = RandomForestClassifier(n_estimators=500, max_features=None,
                                     min_samples_split=5, max_depth=3, min_samples_leaf=1)
    model = DecisionTreeClassifier(criterion='gini', 
                                   max_depth=6, max_features=None,  min_samples_leaf=9, 
                                   min_samples_split=2, 
                                   random_state=10, splitter='random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, random_state=semilla,
                                bootstrap=True, bootstrap_features=False, 
                                max_features=0.9, max_samples=0.95)

    modelo6 = DecisionTreeClassifier(criterion='gini', max_depth=6, max_features=None, 
                                     min_samples_leaf=9, min_samples_split=2, 
                                     random_state=10, splitter='random')
    
    estimadores = [('Extratrees', modelo3), ('Random Forest', modelo4), 
                   ('Bagging', modelo5), ('Decision tree', modelo6)]
    
    super_learner = SuperLearner(folds=10, random_state=semilla, verbose=2)
    super_learner.add(estimadores)
    super_learner.add(estimadores)
    estimador_final = ExtraTreesClassifier(n_estimators=100, max_features=None,
                                           bootstrap=False, max_depth=11, min_samples_split=4, 
                                           min_samples_leaf=1)
    
    super_learner.add_meta(estimador_final)
    
    super_learner.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(super_learner, X_trn_transformado, Y_trn, cv=kfold, scoring='accuracy')
    
    print("Rendimiento del modelo:") 
    print("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean() * 100, "%") 
    return super_learner


X_trn = X_trn
Y_trn = Y_trn 

modelo_superaprendiz_dos_capas = entrenar_modelo_super_aprendiz_dos_capas(X_trn, Y_trn)


Fitting 3 layers
Processing layer-1             done | 00:00:11
Processing layer-2             done | 00:00:10
Processing layer-3             done | 00:00:00
Fit complete                        | 00:00:22

Fitting 3 layers
Processing layer-1             done | 00:00:12
Processing layer-2             done | 00:00:10
Processing layer-3             done | 00:00:00
Fit complete                        | 00:00:23

Predicting 3 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict complete                    | 00:00:00

Fitting 3 layers
Processing layer-1             done | 00:00:13
Processing layer-2             done | 00:00:11
Processing layer-3             done | 00:00:00
Fit complete                        | 00:00:25

Predicting 3 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict com

In [ ]:
print(modelo_superaprendiz_dos_capas.data)

                          ft-m  ft-s  pt-m  pt-s
layer-1  bagging          2.12  0.13  0.37  0.18
layer-1  decision-tree    0.05  0.04  0.01  0.03
layer-1  extratrees       7.39  0.40  0.69  0.16
layer-1  random-forest    7.28  1.92  0.08  0.04
layer-2  bagging          2.05  0.27  0.16  0.13
layer-2  decision-tree    0.04  0.04  0.00  0.00
layer-2  extratrees       6.59  0.49  0.18  0.07
layer-2  random-forest    6.54  1.50  0.08  0.06



In [ ]:
resultados_df_superaprendiz_dos_capas = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_superaprendiz_dos_capas.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_superaprendiz_dos_capas_entrenamiento = pd.concat([resultados_df_superaprendiz_dos_capas, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_superaprendiz_dos_capas_entrenamiento["MODELO"]='SuperAprendizdoscapas'
resultados_df_superaprendiz_dos_capas_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_superaprendiz_dos_capas_entrenamiento


Predicting 3 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict complete                    | 00:00:00
Precisión:  87.53
Exhaustividad:  86.31
Puntuación F1:  83.66
Exactitud:  86.31
Información Mutua Normalizada (NMI): 58.32
Índice Kappa de Cohen: 71.84


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,87.53,SuperAprendizdoscapas,Entrenamiento
1,Exhaustividad,86.31,SuperAprendizdoscapas,Entrenamiento
2,Puntuación F1,83.66,SuperAprendizdoscapas,Entrenamiento
3,Exactitud,86.31,SuperAprendizdoscapas,Entrenamiento
4,Información Mutua Normalizada (NMI),58.32,SuperAprendizdoscapas,Entrenamiento
5,Índice Kappa de Cohen,71.84,SuperAprendizdoscapas,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_superaprendiz_dos_capas = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_superaprendiz_dos_capas.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_superaprendiz_dos_capas_prueba = pd.concat([resultados_df_superaprendiz_dos_capas, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_superaprendiz_dos_capas_prueba["MODELO"]='SuperAprendizdoscapas'
resultados_df_superaprendiz_dos_capas_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_superaprendiz_dos_capas_prueba


Predicting 3 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict complete                    | 00:00:00
Precisión:  79.0
Exhaustividad:  79.25
Puntuación F1:  76.85
Exactitud:  79.25
Información Mutua Normalizada (NMI): 34.12
Índice Kappa de Cohen: 57.05


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.00,SuperAprendizdoscapas,Prueba
1,Exhaustividad,79.25,SuperAprendizdoscapas,Prueba
2,Puntuación F1,76.85,SuperAprendizdoscapas,Prueba
3,Exactitud,79.25,SuperAprendizdoscapas,Prueba
4,Información Mutua Normalizada (NMI),34.12,SuperAprendizdoscapas,Prueba
5,Índice Kappa de Cohen,57.05,SuperAprendizdoscapas,Prueba


In [ ]:
resultados_df_superaprendiz_dos_capas = pd.concat([resultados_df_superaprendiz_dos_capas_prueba,resultados_df_superaprendiz_dos_capas_entrenamiento], ignore_index=True)
resultados_df_superaprendiz_dos_capas

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.00,SuperAprendizdoscapas,Prueba
1,Exhaustividad,79.25,SuperAprendizdoscapas,Prueba
2,Puntuación F1,76.85,SuperAprendizdoscapas,Prueba
3,Exactitud,79.25,SuperAprendizdoscapas,Prueba
4,Información Mutua Normalizada (NMI),34.12,SuperAprendizdoscapas,Prueba
5,Índice Kappa de Cohen,57.05,SuperAprendizdoscapas,Prueba
6,Precisión,87.53,SuperAprendizdoscapas,Entrenamiento
7,Exhaustividad,86.31,SuperAprendizdoscapas,Entrenamiento
8,Puntuación F1,83.66,SuperAprendizdoscapas,Entrenamiento
9,Exactitud,86.31,SuperAprendizdoscapas,Entrenamiento


### RED NEURONAL

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

def train_neural_network(X_train, y_train, input_size, hidden_size, num_classes, num_epochs=100, learning_rate=0.001):
    # Convertir datos de entrenamiento a tensores de PyTorch
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    
    # Inicializar modelo y función de pérdida
    model = NeuralNetwork(input_size, hidden_size, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Entrenamiento del modelo
    for epoch in range(num_epochs):
        # Forward pass y pérdida
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        
        # Backward pass y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 10 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

    return model

def evaluate_model(model, X_test, y_test):
    # Convertir datos de prueba a tensores de PyTorch
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    # Obtener las predicciones del modelo
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)

    # Calcular las métricas del modelo
    accuracy = accuracy_score(y_test_tensor.numpy(), predicted.numpy())
    precision = precision_score(y_test, predicted.numpy(), average='weighted')
    recall = recall_score(y_test, predicted.numpy(), average='weighted')
    f1 = f1_score(y_test, predicted.numpy(), average='weighted')

    print('Accuracy: {:.2f}%'.format(accuracy * 100))
    print("Precisión: ", precision)
    print("Exhaustividad (Recall): ", recall)
    print("Puntuación F1: ", f1)

X_train_array = X_trn.values
X_test_array = X_tst.values
Y_trn = Y_trn 
input_size = X_trn.shape[1]
num_classes = 4
hidden_size = 170

# Supongamos que X_train, y_train, X_test, y_test son tus datos de entrenamiento y prueba
modelo_red_reuronal = train_neural_network(X_train_array , Y_trn, input_size, hidden_size, num_classes)
evaluate_model(modelo_red_reuronal, X_test_array , Y_tst)


Epoch [10/100], Loss: 1.0089
Epoch [20/100], Loss: 0.7642
Epoch [30/100], Loss: 0.6565
Epoch [40/100], Loss: 0.5998
Epoch [50/100], Loss: 0.5676
Epoch [60/100], Loss: 0.5452
Epoch [70/100], Loss: 0.5278
Epoch [80/100], Loss: 0.5133
Epoch [90/100], Loss: 0.5009
Epoch [100/100], Loss: 0.4900
Accuracy: 79.87%
Precisión:  0.7868495432764301
Exhaustividad (Recall):  0.7987421383647799
Puntuación F1:  0.7817955115764172


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, normalized_mutual_info_score, cohen_kappa_score

# Predecir las etiquetas para los datos de entrenamiento
resultados_df_red_reuronal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

# Convertir datos de entrenamiento a tensores de PyTorch
X_train_tensor = torch.tensor(X_train_array, dtype=torch.float32)

# Obtener las predicciones del modelo
outputs = modelo_red_reuronal(X_train_tensor)
_, predicted = torch.max(outputs, 1)
Y_pred_entrenamiento = predicted.numpy()

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)

print("Precisión: ", round(precision*100, 2))
print("Exhaustividad: ", round(recall*100, 2))
print("Puntuación F1: ", round(f1*100, 2))
print("Exactitud: ", round(accuracy*100, 2))
print("Información Mutua Normalizada (NMI):", round(nmi*100, 2))
print("Índice Kappa de Cohen:", round(kappa*100, 2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi = pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa = pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_red_reuronal_entrenamiento = pd.concat([resultados_df_red_reuronal, df_precision, df_recall, df_f1, df_accuracy, df_nmi, df_kappa], ignore_index=True)
resultados_df_red_reuronal_entrenamiento["MODELO"] = 'RedNeuronal'
resultados_df_red_reuronal_entrenamiento["TIPO_DE_DATOS"] = 'Entrenamiento'

# Imprimir el DataFrame con los resultados
resultados_df_red_reuronal_entrenamiento


Precisión:  78.77
Exhaustividad:  79.37
Puntuación F1:  77.31
Exactitud:  79.37
Información Mutua Normalizada (NMI): 33.16
Índice Kappa de Cohen: 56.6


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,78.77,RedNeuronal,Entrenamiento
1,Exhaustividad,79.37,RedNeuronal,Entrenamiento
2,Puntuación F1,77.31,RedNeuronal,Entrenamiento
3,Exactitud,79.37,RedNeuronal,Entrenamiento
4,Información Mutua Normalizada (NMI),33.16,RedNeuronal,Entrenamiento
5,Índice Kappa de Cohen,56.60,RedNeuronal,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_red_reuronal_prueba = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

# Convertir datos de prueba a tensores de PyTorch
X_test_tensor = torch.tensor(X_test_array, dtype=torch.float32)

# Obtener las predicciones del modelo
outputs = modelo_red_reuronal(X_test_tensor)
_, predicted = torch.max(outputs, 1)
Y_pred_prueba = predicted.numpy()

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)

print("Precisión: ", round(precision*100, 2))
print("Exhaustividad: ", round(recall*100, 2))
print("Puntuación F1: ", round(f1*100, 2))
print("Exactitud: ", round(accuracy*100, 2))
print("Información Mutua Normalizada (NMI):", round(nmi*100, 2))
print("Índice Kappa de Cohen:", round(kappa*100, 2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi = pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa = pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_red_reuronal_prueba = pd.concat([resultados_df_red_reuronal_prueba, df_precision, df_recall, df_f1, df_accuracy, df_nmi, df_kappa], ignore_index=True)
resultados_df_red_reuronal_prueba["MODELO"] = 'RedNeuronal'
resultados_df_red_reuronal_prueba["TIPO_DE_DATOS"] = 'Prueba'

# Imprimir el DataFrame con los resultados
resultados_df_red_reuronal_prueba


Precisión:  78.68
Exhaustividad:  79.87
Puntuación F1:  78.18
Exactitud:  79.87
Información Mutua Normalizada (NMI): 33.46
Índice Kappa de Cohen: 58.58


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,78.68,RedNeuronal,Prueba
1,Exhaustividad,79.87,RedNeuronal,Prueba
2,Puntuación F1,78.18,RedNeuronal,Prueba
3,Exactitud,79.87,RedNeuronal,Prueba
4,Información Mutua Normalizada (NMI),33.46,RedNeuronal,Prueba
5,Índice Kappa de Cohen,58.58,RedNeuronal,Prueba


In [ ]:
resultados_df_red_reuronal = pd.concat([resultados_df_red_reuronal_prueba,resultados_df_red_reuronal_entrenamiento], ignore_index=True)
resultados_df_red_reuronal

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,78.68,RedNeuronal,Prueba
1,Exhaustividad,79.87,RedNeuronal,Prueba
2,Puntuación F1,78.18,RedNeuronal,Prueba
3,Exactitud,79.87,RedNeuronal,Prueba
4,Información Mutua Normalizada (NMI),33.46,RedNeuronal,Prueba
5,Índice Kappa de Cohen,58.58,RedNeuronal,Prueba
6,Precisión,78.77,RedNeuronal,Entrenamiento
7,Exhaustividad,79.37,RedNeuronal,Entrenamiento
8,Puntuación F1,77.31,RedNeuronal,Entrenamiento
9,Exactitud,79.37,RedNeuronal,Entrenamiento


### Métricas

In [ ]:
Metricas_Modelos=pd.concat([resultados_df_knn,resultados_df_svc,resultados_df_tree,
                            resultados_df_gaussian,resultados_df_LDA,resultados_df_BG,
                            resultados_df_random,resultados_df_extra,resultados_df_ADA,
                            resultados_df_GD,resultados_df_XB,resultados_df_CB,
                            resultados_df_LIGHT,resultados_df_voting_hard,resultados_df_voting_soft,
                            resultados_df_stacking_lineal,resultados_df_stacking_nolineal,
                            resultados_df_weighted_voting,resultados_df_superaprendiz,
                            resultados_df_superaprendiz_dos_capas,resultados_df_red_reuronal],axis=0)
Metricas_Modelos = Metricas_Modelos.rename(columns={'MÉTRICA': 'METRICA'})
Metricas_Modelos ['METRICA'] = Metricas_Modelos ['METRICA'].apply(lambda x: unidecode(x))
Metricas_Modelos= Metricas_Modelos[Metricas_Modelos["MODELO"].isin(modelos_seleccionados)]

In [ ]:
modelos_seleccionados

['KNeighbors', 'RedNeuronal']

In [ ]:
Metricas_Modelos

,METRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precision,51.65,KNeighbors,Prueba
1,Exhaustividad,58.87,KNeighbors,Prueba
2,Puntuacion F1,53.60,KNeighbors,Prueba
3,Exactitud,58.87,KNeighbors,Prueba
4,Informacion Mutua Normalizada (NMI),0.09,KNeighbors,Prueba
5,Indice Kappa de Cohen,-0.73,KNeighbors,Prueba
6,Precision,60.19,KNeighbors,Entrenamiento
7,Exhaustividad,64.56,KNeighbors,Entrenamiento
8,Puntuacion F1,59.44,KNeighbors,Entrenamiento
9,Exactitud,64.56,KNeighbors,Entrenamiento


In [ ]:
data_with_columns = Metricas_Modelos.to_dict(orient='records')

diccionario_dataframes = [
        {
            'dataTransformacion': data_with_columns,
            'columnas': Metricas_Modelos.columns.tolist()
        }
    ]
with open("Metricas_Modelos.json", "w") as json_file:
    json.dump({"data": diccionario_dataframes}, json_file, indent=4)

    print("Los DataFrames han sido guardados en 'Metricas_Modelos.json'.")

Los DataFrames han sido guardados en 'Metricas_Modelos.json'.


In [ ]:
Metricas_Modelos.to_csv('Metricas_Modelos.csv',sep="|",index=False,encoding='latin-1')

In [ ]:
Metricas_Modelos

,METRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precision,77.48,KNeighbors,Prueba
1,Exhaustividad,78.62,KNeighbors,Prueba
2,Puntuacion F1,77.73,KNeighbors,Prueba
3,Exactitud,78.62,KNeighbors,Prueba
4,Informacion Mutua Normalizada (NMI),30.98,KNeighbors,Prueba
...,...,...,...,...
7,Exhaustividad,79.37,RedNeuronal,Entrenamiento
8,Puntuacion F1,77.31,RedNeuronal,Entrenamiento
9,Exactitud,79.37,RedNeuronal,Entrenamiento
10,Informacion Mutua Normalizada (NMI),33.16,RedNeuronal,Entrenamiento
